# **README.md**
This replication package accompanies the term project for the course *Text-as-Data/Natural Language Processing for Political Science* (LMU Munich, SS 2025).  Its purpose is to provide a transparent, modular, and reproducible workflow for computational text analysis in political science. The workflow is structured into **phases**, each producing artifacts consumed by the next, ensuring that every step can be replicated and validated independently. This reproduction was tested to be functional by the three principla investigators: B.A Seohyun Yoon, B.A. Yegor Novikov, and M.A. Borna Ardehi

### **Why Phases?**
Breaking the pipeline into phases makes the workflow:
* **Reproducible** – each step is documented and produces concrete outputs.  
* **Auditable** – intermediate data (e.g., metadata, sentence files, gold sets) can be inspected.  
* **Modular** – errors or refinements can be addressed at the correct stage without rerunning everything.  
* **Extendable** – future researchers can plug in alternative dictionaries, models, or scaling methods.

### **5 phases**
* **Phase 1:** Converts collected PDFs into clean UTF-8 text and audits coverage.
* **Phase 2:** Builds metadata, removes boilerplate, and segments the corpus into analyzable sentences.
* **Phase 3:** Constructs and validates dictionaries, then trains and applies supervised classifiers.
* **Phase 4:** Implements scaling with seeds and supervised models to measure projection/protection.
* **Phase 5:** Produces descriptive statistics and visualizations of legitimation patterns over time and across organizations.

---

### **Execution Order**
`1.0 → 1.1 → 2.0 → 2.1 → 2.2 → 2.3 → 3.0 → 3.1 → 3.2 → 3.3 → 3.4 → 3.5 → 4.0 → 4.1 → 4.2 → 4.3 → 5.0 → 5.1 → 5.2`

---

### **Technical Requirements**
* Python 3.9+; Jupyter Notebook/Lab.  
* it is recommended to operate this file in a virtual environment in exact replication of the file.

#### **Core packages (pip installations)**
* **Data/ML:** pandas, numpy, scipy, scikit-learn, joblib
* **Text:** pdfminer.six (PDF → text), spacy, ftfy, chardet
* **Progress/CLI:** tqdm, argparse (stdlib)
* **Viz:** matplotlib, seaborn

##### *Installation*
* pip install numpy 
* pip install pandas 
* pip install matplotlib 
* pip install seaborn 
* pip install scikit-learn
* pip install nltk 
* pip install spacy 
* pip install gensim 
* pip install pyreadr 
* pip install pdfminer.six 
* pip install jupyter 
* pip install ipykernel 

##### *Additional Notes*
* nltk and spacy require downloading language models (nltk.download(...), python -m spacy download en_core_web_sm).
* gensim is used for embeddings / representation.
* pdfminer.six is only needed for the raw text extraction step (Phase 1)
* pyreadr is included since some of the data is converted from or to .RData/.sav formats

#### **Directory constants to set before running**
* input_dir, output_dir for Phase 1.0 conversion.
* CORPUS_DIR (and the UTF‑8 mirror path) in Phase 2.0.
* BASE paths in plotting cells (Phase 5.1/5.2).

#### **Required local files**
* **Dictionaries:** norm.txt, funct.txt, proj.txt, prot.txt (2 sets of each)
* **Seeds:** 4_projseed.txt, 4_protseed.txt

* **Outputs chained across phases:** 
0_metadata.csv, metadata_bp.csv, 1_sentences.csv, documents.csv, 2_golden_text.csv, 3_pred_documents.csv, 4_PPGS.csv, 4_norm_sentences_pp_SUP.csv, 4_org_year_pp_SUP.csv

#### **Hardware:** 
Standard laptop capacity is sufficient for dictionary and supervised classification tasks; however scaling large corpora may require GPU access. The writing and execution of this code was performed on M4 Macbook Air with 10 GPU.

___________________________________________________


### **Phase 1.0 — Converting PDF corpus to `.txt` format**

This step transforms the manually collected annual-report PDFs into plain UTF-8 text files. It creates a reproducible raw text corpus from which all subsequent preprocessing and analysis phases draw.

**Process**

* **Imports required tools**
  * `os` — for file path handling and directory creation  
  * `extract_text` from `pdfminer.high_level` — for extracting text from PDF files (`pip install pdfminer.six`)  

* **Defines input and output directories**
  * `input_dir` — location of the manually collected PDF corpus files  
  * `output_dir` — destination folder for the extracted `.txt` files  

* **Ensures output directory exists**
  * Creates `output_dir` if it does not already exist  

* **Iterates over target years (1975–2025)**
  * Constructs expected PDF filename for each year (e.g., `1975_aupcs.pdf`)  
  * Checks if the PDF exists in `input_dir`  
  * If present:  
    – Extracts text from the PDF  
    – Writes the extracted text to a `.txt` file in `output_dir`  
    – Logs the save action  
  * If absent: logs the missing file  

* **Manual adjustment note**
  * The organisation name in the filename pattern must be edited manually for each dataset  
    – e.g., `f"{year}_NATO.pdf"` for NATO, `f"{year}_UNSC.pdf"` for UNSC  
  * Naming of downloaded corpus files follows a uniform structure for consistency  


In [ ]:
import os
from pdfminer.high_level import extract_text

input_dir = '/Users/bornaardehi/Desktop/TAD/Corpus'
output_dir = '/Users/bornaardehi/Desktop/TAD/Text_Corpus'

os.makedirs(output_dir, exist_ok=True)

for year in range(1975, 2026):
    filename_pdf = f"{year}_aupcs.pdf" # Name changes according to which organization's document is being processed
    filepath_pdf = os.path.join(input_dir, filename_pdf)
    filepath_txt = os.path.join(output_dir, f"{year}_aupcs.txt")

    if os.path.exists(filepath_pdf):
        print(f"Processing {filename_pdf}...")
        try:
            text = extract_text(filepath_pdf)
            with open(filepath_txt, 'w', encoding='utf-8') as f:
                f.write(text)
            print(f"Saved {filepath_txt}")
        except Exception as e:
            print(f"Error processing {filename_pdf}: {e}")
    else:
        print(f"Missing: {filename_pdf}")

### **Phase 1.1 — Summarising available `.txt` files and detecting missing years**

This step audits the converted text corpus to determine coverage across years and organisations. It produces a tabular overview of available documents and identifies missing years.

**Process**

* **Imports required tools**
  * `os` — for listing and handling file paths  
  * `pandas` (`pip install pandas`) — for creating and manipulating tabular data  
  * `Path` from `pathlib` — for safer file name parsing  

* **Defines key constants**
  * `CORPUS_DIR` — directory containing the extracted `.txt` corpus files  
  * `YEAR_RANGE` — full set of valid years to check for each organisation (1970–2024 inclusive)  

* **Iterates over `.txt` files in the corpus folder**
  * Skips any non-`.txt` files  
  * Splits the filename into `org` (organisation) and `year` using the last underscore `_` as a separator  
  * Validates that `year` is numeric and falls within `YEAR_RANGE`  
  * Records organisation–year pairs in a list  

* **Creates a DataFrame and summarises coverage**
  * Groups data by organisation to calculate:  
    – `start_year`: earliest available year  
    – `end_year`: latest available year  
    – `count`: total number of available years  
  * Resets the index for a clean table format  

* **Computes missing years per organisation**
  * For each organisation, determines which years in `YEAR_RANGE` are absent  
  * Stores missing years as a comma-separated list in a new column `missing_years`  

* **Reorders and exports the results**
  * Keeps columns in the order: `org`, `start_year`, `end_year`, `missing_years`, `count`  
  * Saves the summary table as `org_summary.csv` in the working directory  


In [ ]:
import os
import pandas as pd
from pathlib import Path

CORPUS_DIR = "/Users/bornaardehi/Desktop/TAD/Text_Corpus"
YEAR_RANGE = set(range(1970, 2025))    # inclusive 1970–2024

records = []
for fname in os.listdir(CORPUS_DIR):
    if not fname.lower().endswith(".txt"):
        continue
    stem = Path(fname).stem
    if "_" not in stem:
        continue
    org, year_str = stem.rsplit("_", 1)
    if not year_str.isdigit():
        continue
    year = int(year_str)
    if year not in YEAR_RANGE:
        continue
    records.append({"org": org, "year": year})

df = pd.DataFrame(records)

# Summarize findings
summary = (
    df
    .groupby("org")["year"]
    .agg([
        ("start_year", "min"),
        ("end_year",   "max"),
        ("count",      "size")
    ])
    .reset_index()
)

# Compute missing years per org
def missing_years_list(row):
    present = set(df.loc[df.org == row.org, "year"])
    missing = sorted(YEAR_RANGE - present)
    return ",".join(str(y) for y in missing)

summary["missing_years"] = summary.apply(missing_years_list, axis=1)

# Reorder columns
summary = summary[[
    "org",
    "start_year",
    "end_year",
    "missing_years",
    "count"
]]

# Export to csv
out_csv = "org_summary.csv"
summary.to_csv(out_csv, index=False)

### **Extra — Renaming corpus files to enforce uniform naming structure**

This auxiliary step standardises the naming convention of extracted text files. It ensures that all corpus files follow a consistent `ORG_YEAR.txt` pattern, which is critical for downstream parsing.

**Process**

* **Imports required tools**  
  * `os` — for interacting with the file system and renaming files  

* **Defines working directory**  
  * `folder_path` — specifies the location of the extracted `.txt` corpus files  

* **Iterates over all files in the target folder**  
  * Processes only files ending with `_nato.txt` (case-sensitive)  
  * Extracts the `year` by splitting the filename at the underscore `_` and taking the first element  
  * Constructs the new filename in the standardised format: `NATO_{year}.txt`  

* **Renames files in place**  
  * Uses `os.rename()` to change each matching file to the uniform naming convention  
  * Prints a log entry showing the original and new file names for verification  


In [ ]:
import os
folder_path = '/Users/bornaardehi/Desktop/TAD/Text_Corpus'
for filename in os.listdir(folder_path):
    if filename.endswith('_nato.txt'):
        year = filename.split('_')[0]
        new_filename = f'NATO_{year}.txt'
        os.rename(
            os.path.join(folder_path, filename),
            os.path.join(folder_path, new_filename)
        )
        print(f'Renamed: {filename} -> {new_filename}')

### **Phase 2.0 — Constructing the metadata / cleaning & audit pipeline**

This step builds a comprehensive metadata table for all corpus files while ensuring clean UTF-8 text versions. It validates file integrity, detects encoding issues, and produces both a canonical metadata CSV and a per-organisation coverage summary.

**Process**

* **Imports required tools**  
  * `os`, `hashlib`, `json`, `shutil` — for file system operations, hashing, and file copying  
  * `Path` from `pathlib` — for platform-independent file path handling  
  * `pandas` (`pip install pandas`) — for tabular metadata storage and export  
  * `chardet` (`pip install chardet`) — for detecting unknown file encodings  
  * `ftfy` (`pip install ftfy`) — for fixing Unicode anomalies  
  * `tqdm` (`pip install tqdm`) — for progress bars during processing  

* **Defines constants and directory structure**  
  * `CORPUS_DIR` — location of the raw `.txt` corpus files  
  * `YEAR_RANGE` — valid year range (1970–2024 inclusive)  
  * `PATTERN_SPLIT` — expected filename separator between `<ORG>` and `<YEAR>`  
  * `CLEAN_DIR` — UTF-8 mirror directory for cleaned corpus files  
  * `META_CSV` — output path for the metadata table  
  * `COVERAGE_CSV` — optional per-organisation coverage summary table  
  * `LOG_JSON` — JSON file storing rejected files and reasons for audit  

* **Creates UTF-8 mirror folder**  
  * Ensures `CLEAN_DIR` exists for storing normalised text files  

* **Defines helper functions**  
  * `sha1_bytes(data)` — generates a SHA-1 hash fingerprint for byte content  
  * `read_and_clean(path)` — reads file content, detects encoding if needed, fixes Unicode issues, writes a cleaned UTF-8 version, and returns:  
    – clean text  
    – original encoding  
    – cleaned file path  

* **Scans and validates corpus files**  
  * Iterates through all `.txt` files recursively in `CORPUS_DIR`  
  * Skips files that:  
    – contain no `_` in the name  
    – have non-numeric or out-of-range years  
    – these are logged to `LOG_JSON` as rejects  
  * For valid files:  
    – Reads and cleans content  
    – Stores SHA-1 fingerprint, organisation, year, original/cleaned paths, original encoding, and byte size  

* **Builds and exports metadata table**  
  * Creates a `pandas` DataFrame, sorted by organisation, year, and original path  
  * Saves metadata to `META_CSV`  

* **Generates per-organisation coverage summary**  
  * Groups by organisation to calculate:  
    – `start_year`  
    – `end_year`  
    – `count` (total documents)  
    – `distinct_years`  
  * Saves coverage summary to `COVERAGE_CSV`  


In [ ]:
import os, hashlib, json, shutil
from pathlib import Path
import pandas as pd
import chardet, ftfy
from tqdm import tqdm        

CORPUS_DIR   = Path("/Users/bornaardehi/Desktop/TAD/Text_Corpus")
YEAR_RANGE   = set(range(1970, 2025))        # 1970-2024
PATTERN_SPLIT = "_"                          
CLEAN_DIR    = CORPUS_DIR.with_name(CORPUS_DIR.name + "_utf8")
META_CSV     = "0_metadata.csv"              # output path. The naming of the files is the indicator of the steps. Anything prior to creation of metadata is step 0.
COVERAGE_CSV = "org_coverage.csv"            # summary table
LOG_JSON     = "unparsable_files.json"       # keeps rejects for extra chek later

# utf8 mirror folder
CLEAN_DIR.mkdir(parents=True, exist_ok=True)
def sha1_bytes(data: bytes) -> str:
    return hashlib.sha1(data).hexdigest()
def read_and_clean(path: Path):
    rel     = path.relative_to(CORPUS_DIR)
    outpath = CLEAN_DIR / rel
    outpath.parent.mkdir(parents=True, exist_ok=True)
    try:
        text  = path.read_text(encoding="utf-8")
        enc   = "utf-8"
    except UnicodeDecodeError:
        raw   = path.read_bytes()
        guess = chardet.detect(raw)
        enc   = guess.get("encoding") or "binary"
        try:
            text = raw.decode(enc, errors="replace")
        except Exception:
            text = raw.decode("utf-8", errors="replace")
    text = ftfy.fix_text(text, normalization="NFKC")
    outpath.write_text(text, encoding="utf-8")

    return text, enc, outpath

# Metadata:
records, rejects = [], []

for path in tqdm(list(CORPUS_DIR.rglob("*.txt")), desc="Scanning files"):
    stem = path.stem
    if PATTERN_SPLIT not in stem:
        rejects.append({"file": str(path), "reason": "no '_' found"})
        continue

    org, year_str = stem.rsplit(PATTERN_SPLIT, 1)
    if not year_str.isdigit():
        rejects.append({"file": str(path), "reason": "year not numeric"})
        continue

    year = int(year_str)
    if year not in YEAR_RANGE:
        rejects.append({"file": str(path), "reason": f"year {year} out of range"})
        continue

    text, enc, clean_path = read_and_clean(path)
    raw_bytes = text.encode("utf-8")
    sha1      = sha1_bytes(raw_bytes)

    records.append({
        "doc_id":       sha1,                      # files' fingerprint
        "org":          org,
        "year":         year,
        "orig_path":    str(path.relative_to(CORPUS_DIR)),
        "clean_path":   str(clean_path.relative_to(CLEAN_DIR)),
        "orig_encoding":enc,
        "bytes":        len(raw_bytes),
        "sha1":         sha1,
    })

# Save
meta = pd.DataFrame(records).sort_values(["org", "year", "orig_path"])
meta.to_csv(META_CSV, index=False)

# Table output for perorg coverage
coverage = (
    meta.groupby("org")["year"]
        .agg(start_year="min", end_year="max", count="size",
             distinct_years="nunique")
        .reset_index()
)
coverage.to_csv(COVERAGE_CSV, index=False)

### **Phase 2.1 — Boilerplate removal via `0_metadata.csv`**

This step strips residual boilerplate text (headers, footers, repeated banners, metadata clutter) from the cleaned UTF-8 corpus.  
Because automated detection does not catch every case, manual curation is applied to ensure consistent removal across all files.

**Process**

* **Imports required tools**  
  * `re` — for compiling regular expressions to detect boilerplate patterns  
  * `hashlib` — for generating SHA-1 fingerprints of cleaned text  
  * `Path` from `pathlib` — for structured file path handling  
  * `pandas` (`pip install pandas`) — for reading and writing metadata tables  

* **Defines input/output paths**  
  * `META_CSV` — metadata file from Phase 2.0 containing UTF-8 corpus info  
  * `CLEAN_DIR` — directory with UTF-8 cleaned corpus files  
  * `BP_DIR` — new directory for boilerplate-free copies of the cleaned corpus  

* **Specifies boilerplate detection rules (RXES)**  
  * Dot leaders or page-break garbage  
  * All-caps or title-case lines resembling page headers  
  * “Table of Contents” or “Contents” lines  
  * Copyright / ISBN / ISSN markers  
  * Contact details (phone, fax, email, URLs)  
  * Standalone bare URLs  
  * Isolated numeric page numbers  
  * Repeated banners such as “Annual Report” or “REPORT”  

* **Defines filter function**  
  * `keep(line)` — returns `True` if no boilerplate pattern matches the given line  

* **Processes each file from `metadata.csv`**  
  * Reads the UTF-8 copy specified in `clean_path`  
  * Splits content into lines and removes any matching an RXES pattern  
  * Joins the kept lines back into a single cleaned text string  

* **Writes boilerplate-free corpus copy**  
  * Saves each processed file to `BP_DIR` with identical relative path/filename  
  * Ensures subdirectories exist before writing  

* **Updates and saves extended metadata**  
  * Inherits all columns from `metadata.csv`  
  * Adds:  
    – `bp_path`: relative path in the boilerplate-free corpus  
    – `chars_bp`: character count after boilerplate removal  
    – `sha1_bp`: SHA-1 fingerprint of the boilerplate-free text  
  * Saves the updated table to `metadata_bp.csv`  


In [ ]:

import re, hashlib
from pathlib import Path # since we're using path, its important to keep THIS file where the main wd is. we're bypassing input/output
import pandas as pd

META_CSV   = "0_metadata.csv"      # 0_ naming is a reference to keep track of the stage in dataset development                     
CLEAN_DIR  = Path("/Users/bornaardehi/Desktop/TAD/Text_Corpus_utf8")
BP_DIR     = CLEAN_DIR.with_name(CLEAN_DIR.name + "_nobp") 
BP_DIR.mkdir(exist_ok=True, parents=True)

RXES = [
    # (1) dots / page-break 
    re.compile(r'^[\s\.]{7,}$'),
    # (2) ALL-CAP or Title-Case lines that look like page headers
    re.compile(r'^[A-ZÁÉÍÓÚÜÑ0-9 ,;\'\-/]{10,}$'), # some files were titled as such. this seleciton was made after going through what we had
    # (3) “Table of Contents” & variants
    re.compile(r'^\s*(TABLE OF CONTENTS|CONTENTS)\s*$', re.I),
    # (4) copyright / ISBN / ISSN / © lines
    re.compile(r'^\s*[©]\s*'),
    re.compile(r'^\s*ISBN\b',  re.I),
    re.compile(r'^\s*ISSN\b',  re.I),
    # (5) contact blocks (Tel, Fax, E-mail, URL)
    re.compile(r'^\s*(Tel|Telephone|Fax|E-?mail|URL)\b', re.I),
    # (6) URLs
    re.compile(r'^\s*https?://', re.I),
    # (7) single line page numbers
    re.compile(r'^\s*\d{1,4}\s*$'),
    # (8) repeated “Annual Report” / “REPORT” banners
    re.compile(r'^\s*(ANNUAL\s+REPORT|REPORT)\s*$', re.I),
]

def keep(line: str) -> bool:
    """Return True if the line should be kept (i.e. not boiler-plate)."""
    return not any(rx.search(line) for rx in RXES)

sha1 = lambda s: hashlib.sha1(s.encode()).hexdigest()

meta = pd.read_csv(META_CSV)
out_rows = []

for _, row in meta.iterrows():
    in_path  = CLEAN_DIR / row.clean_path
    raw_lines = in_path.read_text(encoding="utf-8", errors="ignore").splitlines()

    kept_lines = [ln for ln in raw_lines if keep(ln)]
    cleaned    = "\n".join(kept_lines).strip()

    bp_relpath = Path(row.clean_path)
    out_path   = BP_DIR / bp_relpath
    out_path.parent.mkdir(parents=True, exist_ok=True)
    out_path.write_text(cleaned, encoding="utf-8")

    out_rows.append({
        **row,
        "bp_path":  str(bp_relpath),
        "chars_bp": len(cleaned),
        "sha1_bp":  sha1(cleaned),
    })

pd.DataFrame(out_rows).to_csv("metadata_bp.csv", index=False) # this file was later removed as we had to perform manual clean up and discarded this.

### **Phase 2.2 — Segmenting corpus into sentences**

This step splits the cleaned corpus into individual sentences. It ensures that each sentence is stored as a separate unit, enabling precise downstream classification, scaling, and statistical analysis.

**Process**

* **Imports required tools**  
  * `re` — for pre-sentencization text repairs (hyphenation, spacing)  
  * `Path` from `pathlib` — for file path handling  
  * `pandas` (`pip install pandas`) — for metadata loading and sentence table output  
  * `spacy` (`pip install spacy`; download model with `python -m spacy download en_core_web_sm`) — for NLP-based sentence splitting  
  * `tqdm` (`pip install tqdm`) — for progress tracking  

* **Defines configuration variables**  
  * `META_CSV` — metadata file from Phase 2.0 containing cleaned UTF-8 file paths  
  * `UTF8_DIR` — folder containing UTF-8 cleaned corpus files  
  * `OUT_CSV` — output file for sentence-level corpus  

* **Initialises spaCy with sentencizer only**  
  * Loads `en_core_web_sm` with all heavy components disabled (no POS tagging, parsing, NER) for speed  
  * Adds `sentencizer` pipeline component if missing  
  * Sets initial `nlp.max_length` to 3 million characters (adjustable per document)  

* **Compiles regex patterns for text repair**  
  * `RE_LINE_HYPHEN`: joins words split across lines (`inter-\naction → interaction`)  
  * `RE_NEWLINES`: replaces newlines with spaces  
  * `RE_POLICY_HYPHEN`: preserves compound policy terms by replacing hyphen with underscore (`peace-building → peace_building`)  
  * `RE_SPACES`: collapses multiple spaces into one  

* **Defines repair function**  
  * `repair_text(txt)` applies the regex patterns in sequence  
  * Converts text to lowercase and strips whitespace  

* **Iterates through corpus metadata**  
  * Reads each UTF-8 cleaned file based on `clean_path`  
  * Applies `repair_text()` to prepare content for segmentation  
  * Dynamically increases `nlp.max_length` if document exceeds current limit  
  * Uses spaCy to split text into sentences  

* **Collects sentence-level data**  
  * Skips empty sentences  
  * For each sentence, records:  
    – `doc_id` (hash from Phase 2.0)  
    – `org` (organisation)  
    – `year` (document year)  
    – `sent_id` (sentence number within doc)  
    – `sentence` (clean sentence text)  

* **Saves output**  
  * Exports all sentence records to `sentences.csv` for downstream analysis  


In [ ]:
import re
from pathlib import Path
import pandas as pd
import spacy
from tqdm import tqdm

META_CSV   = "0_metadata.csv"                            # produced earlier
UTF8_DIR   = Path("/Users/bornaardehi/Desktop/TAD/Text_Corpus_utf8")
OUT_CSV    = "1_sentences.csv"                           # final output (there is one more iteration after this in the next step)

nlp = spacy.load("en_core_web_sm",
                 disable=["tagger", "parser", "attribute_ruler",
                          "lemmatizer", "ner"])
if not nlp.has_pipe("sentencizer"):
    nlp.add_pipe("sentencizer")

nlp.max_length = 3_000_000

# REGEXES 
RE_LINE_HYPHEN   = re.compile(r"-\s*\n\s*")
RE_NEWLINES      = re.compile(r"\s*\n\s*")
RE_POLICY_HYPHEN = re.compile(r"(\b\w+)-(\w+\b)")
RE_SPACES        = re.compile(r"\s{2,}")

def repair_text(txt: str) -> str:
    txt = RE_LINE_HYPHEN.sub("", txt)
    txt = RE_NEWLINES.sub(" ", txt)
    txt = RE_POLICY_HYPHEN.sub(r"\1_\2", txt)
    txt = RE_SPACES.sub(" ", txt)
    return txt.lower().strip()

meta = pd.read_csv(META_CSV)

rows = []
for _, m in tqdm(meta.iterrows(), total=len(meta), desc="Sentence splitting"):
    path = UTF8_DIR / m["clean_path"]
    text = path.read_text(encoding="utf-8", errors="ignore")

    text = repair_text(text)
    if len(text) >= nlp.max_length:
        nlp.max_length = len(text) + 1

    doc = nlp(text)

    for sid, sent in enumerate(doc.sents, start=1):
        sent_txt = sent.text.strip()
        if not sent_txt:
            continue
        rows.append({
            "doc_id":   m["doc_id"],
            "org":      m["org"],
            "year":     m["year"],
            "sent_id":  sid,
            "sentence": sent_txt
        })

pd.DataFrame(rows).to_csv(OUT_CSV, index=False)

### **Phase 2.3 — Re-running preprocessing to produce a cleaner sentence DataFrame**

This step refines the sentence-level corpus by removing OCR artifacts, enumeration tokens, and underspecified text fragments. The result is the `1_documents.csv` file that serves as the working dataset for supervised and scaling phases.

**Process**

* **Imports required tools**  
  * `re` — for advanced text cleaning with regular expressions  
  * `pandas` (`pip install pandas`) — for loading and saving CSV files  

* **Defines input/output paths**  
  * `IN_CSV` — the sentence-level corpus produced in Phase 2.2 (`1_sentences.csv`)  
  * `OUT_CSV` — the cleaned output file containing only well-formed sentences (`1_documents.csv`)  

* **Compiles regex patterns**  
  * `RE_SPACED`: detects OCR drift patterns where letters are spaced individually (`t h e p r o j e c t`)  
  * `RE_ENUM`: matches enumeration markers such as `1.`, `6.2.1.`, or `b.` at the start of a token  

* **Defines helper functions**  
  * `collapse(match)` — removes spaces from spaced-letter OCR fragments  
  * `clean_sentence(text)` — applies multiple cleaning rules:  
    – Collapses spaced letters  
    – Removes enumeration tokens and single-character tokens  
    – Collapses multiple spaces to one  
    – Requires at least 4 alphabetic words (each ≥ 2 letters) for a sentence to be kept; otherwise returns an empty string  

* **Processes sentence-level corpus**  
  * Loads `1_sentences.csv` as a DataFrame (all columns read as strings)  
  * Applies `clean_sentence()` to each sentence  
  * Drops any sentences reduced to empty strings after cleaning  

* **Saves cleaned output**  
  * Writes the cleaned DataFrame to `documents.csv` for downstream analysis  


In [ ]:
import re, pandas as pd

IN_CSV  = "1_sentences.csv"           # from last step, as mentioned, we reproduced an alternative after manual boilerplate removal of all documents.
OUT_CSV = "1_documents.csv"           # the final metadata for replciaiton, after multiple clean up iterations

# regexes 
# spaced OCR drift: sequence of single letters separated by spaces
RE_SPACED = re.compile(r'(?:\b[a-zA-Z]\s+){2,}[a-zA-Z]\b') # based on the cases we noticed requiring removal
# enumeration tokens:  1.   6.2.1.   b.
RE_ENUM = re.compile(r'^([a-zA-Z]|\d+)(?:\.(?:[a-zA-Z]|\d+))*\.?$') # also based on the cases we noticed requiring removal

def collapse(match):
    """Remove spaces in spaced-letter chunk."""
    return match.group(0).replace(" ", "")

def clean_sentence(text: str) -> str:
    # 1) collapse spaced letters
    text = RE_SPACED.sub(collapse, text)

    # 2) remove enumeration tokens & 1-letter tokens
    tokens = [t for t in text.split() if not RE_ENUM.match(t) and len(t) > 1]
    if not tokens:
        return ""

    text = " ".join(tokens)
    text = re.sub(r'\s{2,}', ' ', text).strip()

    # 3) require at least 4 alphabetic words (length ≥2)
    alpha_words = re.findall(r'[a-zA-Z]{2,}', text)
    if len(alpha_words) < 4:
        return ""

    return text

df = pd.read_csv(IN_CSV, dtype=str)
df["sentence"] = df["sentence"].apply(clean_sentence)
df = df[df["sentence"].str.len() > 0]

df.to_csv(OUT_CSV, index=False)

### **Phase 2.3 –– Dictionary construction**
We built the dictionaries for the supervised dictionary SVM method. called dict_v1. It included four dictionaries for: normative, functional, projection and protection

### **Phase 3.1 — Stratified sampling for initial dictionary-based manual labelling**

This phase serves as the bridge between preprocessing and supervised model training. It draws a stratified sample of sentences to be manually coded, creating the initial gold-standard dataset for validating and refining dictionaries.

**Process**

* **Imports required tools**  
  * `pandas` (`pip install pandas`) — for data handling and CSV I/O  
  * `numpy` (`pip install numpy`) — for numerical operations and rounding  
  * `Path` from `pathlib` — for file path management (not essential here, but consistent with workflow)  

* **Defines configuration variables**  
  * `SENT_FILE` — cleaned, sentence-level dataset from Phase 2.3 (`documents.csv`)  
  * `TOTAL_SAMPLE` — total number of sentences to sample (15,000)  
  * `PER_CODER` — number of sentences assigned to each coder (5,000)  
  * `CODER_IDS` — list of unique identifiers for coders: `SH` (Seohyun), `YN` (Yegor), `BA` (Borna)  
  * `RANDOM_STATE` — fixed seed for reproducibility of sampling  

* **Loads dataset and defines strata**  
  * Reads `documents.csv` into a DataFrame  
  * Creates a `decade` column from `year` for stratification (e.g., `1983 → 1980`)  
  * Groups data into strata by `org` and `decade`  

* **Performs stratified sampling**  
  * For each `(org, decade)` group, calculates proportional sample size relative to dataset share  
  * Ensures sample size does not exceed group size  
  * Samples rows from each group using fixed random seed  
  * Concatenates all strata samples into one DataFrame, shuffles once, and resets index  
  * If total exceeds `TOTAL_SAMPLE`, down-samples to exactly that size  

* **Splits sample into coder-specific files**  
  * Divides sampled DataFrame into consecutive blocks of `PER_CODER` rows  
  * Adds an empty `label` column after the sentence column for coders to fill  
  * Writes each block to a separate file: `{CODER_ID}_to_label.csv`  
  * Logs the number of rows in each output file  

* **Outcome**  
  * Three CSV files generated:  
    – `SH_to_label.csv`  
    – `YN_to_label.csv`  
    – `BA_to_label.csv`  
  * Each file contains 5,000 randomly selected, stratified sentences, ready for human coding in the first iteration  


In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path

SENT_FILE      = "1_documents.csv"
TOTAL_SAMPLE   = 15_000 # choesn based on recommendations from the slides, to utilize at least 5% of the total corpus for training
PER_CODER      = 5_000
CODER_IDS      = ["SH", "YN", "BA"] # three PIs
RANDOM_STATE   = 42                   # reproducible

sent = pd.read_csv(SENT_FILE)
sent["decade"] = (sent["year"] // 10) * 10      # 1970→1970, 1983→1980…
strata = sent.groupby(["org", "decade"])

# stratified sampling is essentially randomised, which allows for greater empirical access to variety of sentences we're training on 
samples = []
for _, group in strata:
    # proportional allocation
    n = int(np.round(len(group) / len(sent) * TOTAL_SAMPLE))
    n = min(n, len(group))
    if n > 0:
        samples.append(group.sample(n=n, random_state=RANDOM_STATE))

sampled = (pd.concat(samples)
             .sample(frac=1, random_state=RANDOM_STATE)   # shuffle once
             .reset_index(drop=True))

# exactly TOTAL_SAMPLE rows
if len(sampled) > TOTAL_SAMPLE:
    sampled = sampled.sample(n=TOTAL_SAMPLE, random_state=RANDOM_STATE).reset_index(drop=True)

# split into coder files 
for i, coder in enumerate(CODER_IDS):
    start = i * PER_CODER
    end   = start + PER_CODER
    part  = sampled.iloc[start:end].copy()
    part.insert(part.columns.get_loc("sentence") + 1, "label", "")  # blank label col
    outfile = f"{coder}_to_label.csv"
    part.to_csv(outfile, index=False)


### **Phase 3.2 — Second iteration: computer-assisted labelling with updated dictionaries**

In this iteration, the term dictionaries (`funct.txt`, `norm.txt`, `proj.txt`, `prot.txt`) were manually revised before use.  
Some words were **ablated (removed)** due to high false-positive rates in Phase 3.1, often because of polysemy (same term used in unrelated contexts) or excessive overmatching that created noise in coder workload.  

This phase reduces manual labelling burden by pre-filtering the corpus with a tuned dictionary, ensuring coders see only sentences likely relevant to the classification task, while balancing coverage across organisations and decades.

**Process**

* **Imports required tools**  
  * `pandas` (`pip install pandas`) — for corpus handling and sampling  
  * `numpy` (`pip install numpy`) — for proportional allocation  
  * `re` — for token pattern matching  
  * `sys` — for clean script exits on validation errors  
  * `Path` from `pathlib` — for file path handling  

* **Defines key file paths**  
  * `DOC_CSV` — cleaned sentence-level corpus from Phase 2.3 (`1_documents.csv`)  
  * `DICT_DIR` — directory containing revised dictionary files  
  * `OUT_GOLD` — output file for the 15,000-sentence gold sample (`virgin_text.csv`)  
  * `CODERS` — list of human coders: `SH`, `BA`, `YN`  

* **Loads and validates corpus**  
  * Ensures `documents.csv` contains required columns: `doc_id`, `org`, `year`, `sent_id`, `sentence`  
  * Drops empty or whitespace-only sentences  

* **Loads dictionaries**  
  * Reads each dictionary file into a set of lowercase terms, skipping blanks  
  * Validates that all dictionary files exist before proceeding  

* **Flags sentences containing dictionary terms**  
  * Tokenises each sentence into lowercase words  
  * For each dictionary, creates a binary flag (1 if at least one term present, else 0)  
  * Appends these flags as new columns in the DataFrame  

* **Filters corpus to relevant sentences**  
  * Retains only sentences containing ≥1 dictionary term across all four dictionaries  
  * Validates that enough candidates exist to build a 15,000-sentence sample  

* **Draws stratified sample**  
  * Creates decade strata from `year` and groups by `(org, decade)`  
  * Allocates sample size proportionally to each stratum’s share of the filtered corpus  
  * Adjusts for overshoot (random down-sample) or undershoot (fills from remainder pool)  

* **Saves the gold sample**  
  * Writes the final stratified 15,000-sentence sample to `virgin_text.csv`  

* **Prepares coder verification files**  
  * Shuffles the gold sample  
  * Splits into three equal chunks of 5,000 sentences each  
  * Inserts an empty `label` column after the sentence column  
  * Saves each chunk as `verify_{CODER}.csv` for manual verification  


In [ ]:
import pandas as pd, numpy as np, re, sys
from pathlib import Path

BASE      = Path(__file__).resolve().parent        
DOC_CSV   = BASE / "documents.csv"
DICT_DIR  = BASE / "dict_v1"                           # contains the four initial dictionaries
OUT_GOLD  = BASE / "virgin_text.csv"
CODERS    = ["SH", "BA", "YN"]

df = pd.read_csv(DOC_CSV)

# ensure mandatory cols exist
req_cols = {"doc_id", "org", "year", "sent_id", "sentence"}
if not req_cols.issubset(df.columns):
    sys.exit(f"✗ {DOC_CSV} must contain columns: {', '.join(sorted(req_cols))}")

# drop whitespace sentences
df["sentence"] = df["sentence"].fillna("").astype(str)
df = df[df["sentence"].str.strip().astype(bool)]

DICT_FILES = {"funct": "funct.txt",
              "norm":  "norm.txt",
              "proj":  "proj.txt",
              "prot":  "prot.txt"}

DICTS = {}
for key, fname in DICT_FILES.items():
    fpath = DICT_DIR / fname
    if not fpath.exists():
        sys.exit(f"✗ dictionary file missing: {fpath}")
    DICTS[key] = {ln.strip().lower() for ln in fpath.open(encoding="utf-8")
                  if ln.strip()}

tok_pat = re.compile(r"\b\w+\b")

def flag_row(text: str):
    toks = tok_pat.findall(text.lower())
    return {k: int(any(t in vocab for t in toks)) for k, vocab in DICTS.items()}

flags_df = df["sentence"].apply(flag_row).apply(pd.Series)
df = pd.concat([df, flags_df], axis=1)

df = df[df[list(DICT_FILES)].sum(axis=1) > 0]

if len(df) < 15_000:
    sys.exit("✗ Not enough labelled sentences to reach 15 000")

# STRATIFIED 15 000-ROW SAMPLE
df["decade"] = (df["year"] // 10) * 10
strata = df.groupby(["org", "decade"])

target = 15_000
rows = []

for _, grp in strata:
    # proportional allocation
    n = int(round(len(grp) / len(df) * target))
    n = min(n, len(grp))
    if n:
        rows.append(grp.sample(n=n, random_state=42))

sample = pd.concat(rows).reset_index(drop=True)

# adjust if overshoot / undershoot
if len(sample) > target:
    sample = sample.sample(n=target, random_state=42)
elif len(sample) < target:
    # randomly add extra rows from remaining pool
    remainder = df.drop(sample.index)
    extra = remainder.sample(n=target - len(sample), random_state=42)
    sample = pd.concat([sample, extra]).reset_index(drop=True)


sample.to_csv(OUT_GOLD, index=False)

sample_shuffled = sample.sample(frac=1, random_state=99).reset_index(drop=True)
chunk = 5_000

for i, coder in enumerate(CODERS):
    part = sample_shuffled.iloc[i*chunk:(i+1)*chunk].copy()
    part.insert(part.columns.get_loc("sentence") + 1, "label", "")   # blank label
    out = BASE / f"verify_{coder}.csv"
    part.to_csv(out, index=False)

### **Phase 3.3 — Third iteration: reduced dictionary size and refined subset selection**

In this iteration, the dictionaries were deliberately reduced to remove marginal, ambiguous, or low-precision terms that inflated earlier samples with irrelevant content.  
Only **two categories** were retained — **functional (`funct.txt`)** and **normative (`norm.txt`)** — simplifying classification and improving coder efficiency.  

**Process**

* **Imports required tools**  
  * `pandas` (`pip install pandas`) — for dataset manipulation and CSV export  
  * `numpy` (`pip install numpy`) — for proportional sampling calculations  
  * `re` — for tokenisation  
  * `sys` — for clean exits on missing files or validation errors  
  * `Path` from `pathlib` — for structured file path handling  

* **Defines file paths**  
  * `DOC_CSV` — cleaned sentence-level corpus from Phase 2.3 (`documents.csv`)  
  * `DICT_DIR` — directory containing reduced dictionaries (`funct.txt`, `norm.txt`)  
  * `OUT_CSV` — output file for the final refined sample (`golden_text.csv`)  

* **Loads and validates corpus**  
  * Ensures `documents.csv` exists and contains required columns: `doc_id`, `org`, `year`, `sent_id`, `sentence`  
  * Removes empty or whitespace-only sentences  

* **Loads reduced dictionaries**  
  * `load_dict(name)` — reads a dictionary file into a lowercase term set, skipping blanks  
  * Loads `vocab_funct` and `vocab_norm`  

* **Flags and labels sentences**  
  * Tokenises each sentence into lowercase words  
  * Flags **functional** and **normative** matches separately  
  * Assigns label:  
    – `"funct"` if only functional terms matched  
    – `"norm"` if only normative terms matched  
    – `"x"` if both categories matched  
    – `"?"` if no match  
  * Retains only sentences with labels `"funct"`, `"norm"`, or `"x"`  

* **Draws stratified sample**  
  * Groups valid sentences by `(org, decade)`  
  * Allocates sample size proportionally to each stratum’s share of the valid corpus  
  * Ensures each stratum sample does not exceed available rows  
  * Concatenates strata, then trims or tops up to exactly **10,000 sentences**  

* **Exports refined dataset**  
  * Saves the stratified **10,000-sentence sample** to `golden_text.csv` for final analysis or coder verification  


In [ ]:
import pandas as pd, numpy as np, re, sys
from pathlib import Path

BASE       = Path(__file__).resolve().parent        
DOC_CSV    = BASE / "documents.csv"
DICT_DIR   = BASE / "dict_V2"                        # second dictionary
OUT_CSV    = BASE / "golden_text.csv"

if not DOC_CSV.exists():
    sys.exit(f"✗ {DOC_CSV} missing")

df = pd.read_csv(DOC_CSV)
needed = {"doc_id","org","year","sent_id","sentence"}
if not needed.issubset(df.columns):
    sys.exit(f"✗ {DOC_CSV} must contain columns: {', '.join(sorted(needed))}")

df["sentence"] = df["sentence"].fillna("").astype(str)
df = df[df["sentence"].str.strip().astype(bool)]

def load_dict(name):
    f = DICT_DIR / f"{name}.txt"
    if not f.exists():
        sys.exit(f"✗ dictionary file missing: {f}")
    return {ln.strip().lower() for ln in f.open(encoding="utf-8") if ln.strip()}

vocab_funct = load_dict("funct")
vocab_norm  = load_dict("norm")

token_pat = re.compile(r"\b\w+\b")

def flag_row(text):
    toks = token_pat.findall(text.lower())
    fc = int(any(t in vocab_funct for t in toks))
    nc = int(any(t in vocab_norm  for t in toks))
    if fc and not nc:
        lbl = "funct"
    elif nc and not fc:
        lbl = "norm"
    elif fc and nc:
        lbl = "x"
    else:
        lbl = "?"
    return fc, nc, lbl

flags = df["sentence"].apply(flag_row).tolist()
df[["funct","norm","label"]] = pd.DataFrame(flags, index=df.index)

df_valid = df[df["label"] != "?"].reset_index(drop=True)
if len(df_valid) < 10000:
    sys.exit("✗ Not enough labelled sentences to reach 10000 rows.")
df_valid["decade"] = (df_valid["year"] // 10) * 10
strata = df_valid.groupby(["org","decade"])

target = 10000
rows = []
for _, grp in strata:
    n = int(round(len(grp) / len(df_valid) * target))
    n = min(n, len(grp))
    if n:
        rows.append(grp.sample(n=n, random_state=7))

sample = pd.concat(rows).reset_index(drop=True)

if len(sample) > target:
    sample = sample.sample(n=target, random_state=7).reset_index(drop=True)
elif len(sample) < target:
    extra = (df_valid
             .drop(sample.index)
             .sample(n=target - len(sample), random_state=7))
    sample = pd.concat([sample, extra]).reset_index(drop=True)

assert len(sample) == 10000, "Sample is not 100 rows!"

# ── 5. EXPORT -------------------------------------------------
sample.to_csv(OUT_CSV, index=False)

### **Phase 3.4 — Supervised classification: functional vs normative + Model and Vector trainer**

This phase uses the refined **10,000-row golden dataset** from Phase 3.3 to train and validate a binary classifier distinguishing **functional** from **normative** sentences.  
Dictionary matches are included as explicit features alongside TF–IDF vectors to boost interpretability and performance.

**Process**

* **Imports required tools**  
  * `re` — for tokenisation and dictionary matching  
  * `joblib` (`pip install joblib`) — for saving trained models  
  * `Path` from `pathlib` — for file path handling  
  * `numpy` (`pip install numpy`) — for numeric array handling  
  * `pandas` (`pip install pandas`) — for dataset handling  
  * `scipy.sparse` (`pip install scipy`) — for handling sparse matrices  
  * `sklearn.feature_extraction.text.TfidfVectorizer` — for text vectorisation  
  * `sklearn.metrics.confusion_matrix` — for evaluation  
  * `sklearn.model_selection` (`GroupKFold`, `GridSearchCV`, `cross_val_predict`) — for grouped cross-validation and hyperparameter tuning  
  * `sklearn.svm.LinearSVC` — for linear SVM classification  

* **Defines file paths**  
  * `DICT_D` — directory containing reduced dictionaries (`funct.txt`, `norm.txt`)  
  * `GOLD_CSV` — 10k-sentence golden dataset from Phase 3.3  
  * `MODEL_DIR` — output folder for trained models and artifacts  

* **Loads dictionaries**  
  * `load_vocab(fname)` — reads a dictionary into a lowercase term set  
  * Loads `vocab_funct` and `vocab_norm`  

* **Loads and prepares golden dataset**  
  * Reads `GOLD_CSV` and ensures a `label` column exists  
  * Keeps only rows with labels in `{funct, norm}`  
  * Converts `label` to binary target `y` (`1 = normative`, `0 = functional`)  
  * Stores `doc_id` for grouped cross-validation (ensuring no sentence leakage across folds)  

* **Feature engineering**  
  * `dict_flags(text)` — produces two binary features per sentence:  
    – `funct_cnt`: at least one functional term present  
    – `norm_cnt`: at least one normative term present  
  * Generates a `dict_df` DataFrame of these features  
  * Computes TF–IDF vectors with:  
    – unigrams and bigrams (`ngram_range=(1, 2)`)  
    – terms appearing in ≥20 documents and ≤90% of documents  
  * Concatenates TF–IDF matrix with dictionary-flag features into a final sparse feature matrix `X`  

* **Model selection via cross-validation**  
  * Uses `GroupKFold(n_splits=10)` for grouped CV (by `doc_id`)  
  * Runs `GridSearchCV` over `LinearSVC(class_weight="balanced")` with `C ∈ {0.1, 0.5, 1, 3, 10}`  
  * Selects the `C` value with the highest mean F1 score  
  * Produces cross-validated predictions for confusion matrix analysis  

* **Final model training and saving**  
  * Trains the best `LinearSVC` on the full dataset  
  * Saves artifacts to `MODEL_DIR`:  
    – `svm_norm_funct.joblib` (trained SVM)  
    – `tfidf_vectorizer.joblib` (TF–IDF vectoriser)  
    – `gold_dict_features.csv` (dictionary match features for the golden set)  


In [ ]:
import re, joblib
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import (GroupKFold, GridSearchCV,
                                     cross_val_predict)
from sklearn.svm import LinearSVC

BASE    = Path(__file__).resolve().parent          
DICT_D  = BASE / "dict_v2"                         
GOLD_CSV = BASE / "2_golden_text.csv"              


def load_vocab(fname: str):
    path = DICT_D / fname
    if not path.exists():
        raise FileNotFoundError(f"Dictionary file missing: {path}")
    return {ln.strip().lower() for ln in path.open(encoding="utf-8")
            if ln.strip()}

vocab_funct = load_vocab("funct.txt")
vocab_norm  = load_vocab("norm.txt")

gold = pd.read_csv(GOLD_CSV)
if "label" not in gold.columns:
    raise ValueError("'label' column not found in golden file")

gold = gold[gold["label"].isin(["funct", "norm"])].reset_index(drop=True)
gold["y"] = (gold["label"] == "norm").astype(int)    # 1 = Normative

token_pat = re.compile(r"\b\w+\b")

def dict_flags(text: str):
    toks = token_pat.findall(text.lower())
    return [
        int(any(t in vocab_funct for t in toks)),     # funct_cnt
        int(any(t in vocab_norm  for t in toks)),     # norm_cnt
    ]

flags = np.array([dict_flags(t) for t in gold["sentence"]])
dict_df = pd.DataFrame(flags, columns=["funct_cnt", "norm_cnt"])

# TF-IDF matrix
tfidf = TfidfVectorizer(ngram_range=(1, 2),
                        min_df=20,
                        max_df=0.9)
X_tfidf = tfidf.fit_transform(gold["sentence"])

# merge sparse + dense
X = hstack([X_tfidf, csr_matrix(flags)])
y = gold["y"].values
groups = gold["doc_id"]              # group CV by document


param_grid = {"C": [0.1, 0.5, 1, 3, 10]}
svm = LinearSVC(class_weight="balanced")

cv = GroupKFold(n_splits=10)
gs = GridSearchCV(svm, param_grid, cv=cv,
                  scoring="f1", n_jobs=-1, verbose=1)

gs.fit(X, y, groups=groups)

# Confusion matrix on CV predictions
y_pred_cv = cross_val_predict(gs.best_estimator_, X, y, cv=cv, groups=groups)
cm = pd.DataFrame(confusion_matrix(y, y_pred_cv),
                  index=["True_Funct", "True_Norm"],
                  columns=["Pred_Funct", "Pred_Norm"])

best_svm = gs.best_estimator_.fit(X, y)

MODEL_DIR = BASE / "models"
MODEL_DIR.mkdir(exist_ok=True)

joblib.dump(best_svm, MODEL_DIR / "svm_norm_funct.joblib")
joblib.dump(tfidf,    MODEL_DIR / "tfidf_vectorizer.joblib")
dict_df.to_csv(MODEL_DIR / "gold_dict_features.csv", index=False)

### **Phase 3.5 — Applying trained classifier to the full corpus**

This phase loads the trained SVM classifier and TF–IDF vectoriser from Phase 3.4, reconstructs features for the entire cleaned corpus, applies the model to generate predictions, and outputs an auto-labelled dataset for downstream analysis.

**Process**

* **Imports required tools**  
  * `pandas` — for corpus loading and output  
  * `re` — for tokenisation and dictionary matching  
  * `joblib` — for loading persisted models  
  * `numpy` — for array handling  
  * `Path` from `pathlib` — for path handling  
  * `scipy.sparse` — for combining sparse feature matrices  

* **Defines file paths**  
  * `MODEL_D` — directory with saved `svm_norm_funct.joblib` and `tfidf_vectorizer.joblib`  
  * `CORPUS_CSV` — full cleaned corpus at the sentence level (`1_documents.csv`)  
  * `DICT_DIR` — reduced dictionary directory (`funct.txt`, `norm.txt`)  
  * `OUT_CSV` — output file containing corpus plus predicted labels (`3_pred_documents.csv`)  

* **Loads model and vectoriser**  
  * Loads trained `LinearSVC` model from Phase 3.4  
  * Loads `TfidfVectorizer` fitted on the golden set vocabulary  

* **Loads dictionaries and defines feature function**  
  * Reads `funct.txt` and `norm.txt` into term sets  
  * `dict_counts(text)` tokenises a sentence and returns two binary features:  
    – `funct_cnt = 1` if any functional term present  
    – `norm_cnt = 1` if any normative term present  

* **Loads full corpus**  
  * Reads `1_documents.csv` containing all sentences to be classified  

* **Builds feature matrix**  
  * Applies `dict_counts` to each sentence to create a binary feature DataFrame  
  * Transforms sentences into TF–IDF vectors using the loaded vectoriser  
  * Horizontally stacks the TF–IDF matrix with dictionary-flag features into a unified sparse matrix (`X_full`)  

* **Generates predictions**  
  * Uses `svm_model.predict()` to classify each sentence as **Normative (1)** or **Functional (0)**  
  * Stores predictions in new column `pred_normative`  
  * Writes full dataset with predictions to `3_pred_documents.csv`  

* **Performs sanity checks**  
  * Computes proportion of sentences predicted as normative  
  * Samples 10 random sentences from each predicted class for quick qualitative inspection  


In [ ]:

import pandas as pd, re, joblib, numpy as np
from pathlib import Path
from scipy.sparse import csr_matrix, hstack

BASE     = Path(__file__).resolve().parent       
MODEL_D  = BASE / "models"
CORPUS_CSV = BASE / "1_documents.csv"
OUT_CSV    = BASE / "3_pred_documents.csv"
DICT_DIR   = BASE / "dict_V2"
svm_path   = MODEL_D / "svm_norm_funct.joblib"
vec_path   = MODEL_D / "tfidf_vectorizer.joblib"
svm_model  = joblib.load(svm_path)
tfidf_vec  = joblib.load(vec_path)


def load_vocab(fname):
    return {ln.strip().lower()
            for ln in (DICT_DIR/fname).open(encoding="utf-8") if ln.strip()}

vocab_funct = load_vocab("funct.txt")
vocab_norm  = load_vocab("norm.txt")

token_pat = re.compile(r"\b\w+\b")
def dict_counts(text):
    toks = token_pat.findall(text.lower())
    return {
        "funct_cnt": int(any(t in vocab_funct for t in toks)),
        "norm_cnt":  int(any(t in vocab_norm  for t in toks))
    }


df = pd.read_csv(CORPUS_CSV)

dict_df      = df["sentence"].apply(dict_counts).apply(pd.Series)
X_other      = csr_matrix(dict_df.values)
X_tfidf      = tfidf_vec.transform(df["sentence"].astype(str))
X_full       = hstack([X_tfidf, X_other])

df["pred_normative"] = svm_model.predict(X_full)   # 1 = Normative, 0 = Functional
df.to_csv(OUT_CSV, index=False)

share_norm = df["pred_normative"].mean()

# Sample 10 sentences of each class for eyeballing
def sample_text(cls, n=10):
    rows = df[df["pred_normative"]==cls].sample(n=min(n, (df["pred_normative"]==cls).sum()),
                                                random_state=1)["sentence"]
    tag  = "Normative" if cls==1 else "Functional"
    print(f"\n---- Random {len(rows)} {tag} sentences ----")
    for s in rows:
        print("•", s[:200].replace("\n"," ") + ("..." if len(s)>200 else ""))

sample_text(1)   # Normative examples
sample_text(0)   # Functional examples



### **Phase 4.0 –– Handcoded scaling** 

Although we first attempted to hand-code extreme cases for supervised scaling with partial automation assistance, the quality of both the output and the auto-labelling proved problematic. We therefore decided to randomly select corpora with high frequencies of projection and protection to identify our scale items. The result was two seed baskets, stored as 4_projseed.txt and 4_protseed.txt.

These seed sets served as the virgin texts and golden sets on which we trained the projection/protection classifier, which was subsequently applied to the corpus. Since the previous SVM stage had already isolated normative sentences, and our aim was to distinguish projection vs protection within normative content, we used the output of Phase 3.5 as our primary DataFrame for this stage.

### **Phase 4.1 — Applying projection/protection seeds**

This step prepares the projection and protection seed sentences that anchor the supervised scaling process.  
It normalises, labels, and combines the two seed sets into a single reference table used for projection–protection classification.

**Process**

* **Imports required tools**  
  * `pandas` (`pip install pandas`) — for table creation and CSV writing  
  * `unicodedata` — for Unicode normalisation  
  * `Path` from `pathlib` — for file path handling  

* **Defines file paths**  
  * `BASE` — project root directory  
  * `PROJ_TXT` — path to projection seed sentences (`4_projseed.txt`)  
  * `PROT_TXT` — path to protection seed sentences (`4_protseed.txt`)  
  * `OUT_CSV` — output file for combined seeds (`4_PPGS.csv`)  

* **Defines helper functions**  
  * `normalize_sentence(s)` — applies Unicode NFKC normalisation, trims stray quotes/spaces, and collapses multiple spaces into one  
  * `read_sentences(path)` — reads lines from a text file, normalises them, and removes empty entries  

* **Reads and labels seed sentences**  
  * Loads projection seeds from `PROJ_TXT` and labels them `"proj"`  
  * Loads protection seeds from `PROT_TXT` and labels them `"prot"`  

* **Builds DataFrame and saves output**  
  * Combines both labelled sets into a single DataFrame  
  * Removes duplicates based on the `sentence` column  
  * Saves the combined dataset to `4_PPGS.csv` in UTF-8 encoding  

* **Output**  
  * Prints confirmation of output path and row count  
  * Displays the first five rows for verification  


In [ ]:
import pandas as pd
import unicodedata
from pathlib import Path

BASE = Path("/Users/bornaardehi/Desktop/TAD")
PROJ_TXT = BASE / "4_projseed.txt"
PROT_TXT = BASE / "4_protseed.txt"
OUT_CSV  = BASE / "4_PPGS.csv"

def normalize_sentence(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)  # normalize quotes, spaces, etc.
    s = s.strip()
    s = s.strip(' "\'')  # strip stray quotes
    # collapse multiple spaces
    while "  " in s:
        s = s.replace("  ", " ")
    return s

def read_sentences(path: Path):
    """Read non-empty, cleaned lines from a .txt file."""
    with open(path, "r", encoding="utf-8") as f:
        lines = [normalize_sentence(line) for line in f]
    # remove empties
    return [ln for ln in lines if ln]

proj_sentences = read_sentences(PROJ_TXT)
prot_sentences = read_sentences(PROT_TXT)

rows = []
rows.extend({"sentence": s, "label": "proj"} for s in proj_sentences)
rows.extend({"sentence": s, "label": "prot"} for s in prot_sentences)

df = pd.DataFrame(rows)
df = df.drop_duplicates(subset=["sentence"]).reset_index(drop=True)
df.to_csv(OUT_CSV, index=False, encoding="utf-8")

### **Phase 4.2 — Projection vs protection supervised scaling**

This phase trains a supervised model to distinguish **projection** from **protection** within normative sentences, using the manually validated gold seed set from Phase 4.1.  
The trained model is then applied to the normative-only subset of the full corpus (from Phase 3.5), generating both **sentence-level** and **aggregated organisation–year** projection/protection scores for further analysis.

**Process**

* **Imports required tools**  
  * `pandas` (`pip install pandas`) — for dataset manipulation  
  * `Path` from `pathlib` — for file path management  
  * `TfidfVectorizer` from `sklearn.feature_extraction.text` — for converting text into TF–IDF features  
  * `LogisticRegression` from `sklearn.linear_model` — for binary classification  
  * `Pipeline` from `sklearn.pipeline` — for chaining preprocessing and modelling  
  * `cross_val_score` from `sklearn.model_selection` — for cross-validation accuracy estimation  
  * `joblib` — for saving the trained model  

* **Defines file paths**  
  * `CORPUS` — full corpus with SVM-predicted normative/functional labels (`3_pred_documents.csv`)  
  * `GOLDSEEDS` — labelled gold seeds from Phase 4.1 (`4_PPGS.csv`)  
  * `MODEL_DIR` — directory for saving the trained projection/protection model  
  * `OUT_SENT` — sentence-level output file (`4_norm_sentences_pp_SUP.csv`)  
  * `OUT_YEAR` — aggregated organisation–year output file (`4_org_year_pp_SUP.csv`)  

* **Loads and encodes gold seeds**  
  * Reads `4_PPGS.csv` and reports label counts  
  * Encodes labels: `"proj" → 1`, `"prot" → 0`  

* **Trains TF–IDF + Logistic Regression classifier**  
  * Configures `TfidfVectorizer`: unigrams and bigrams, `min_df=2`, `max_df=0.9`, lowercasing  
  * Configures `LogisticRegression`: balanced class weights, `liblinear` solver  
  * Wraps both into a `Pipeline`  
  * Runs **5-fold cross-validation** on the gold seeds, printing mean accuracy ± std  
  * Fits the model on the full seed set  
  * Saves the trained pipeline to `pp_scaler.joblib`  

* **Filters normative sentences in the corpus**  
  * Loads full corpus from Phase 3.5 (`3_pred_documents.csv`)  
  * Keeps only rows where `pred_normative == 1`  

* **Scores projection/protection probabilities**  
  * Applies the trained model to normative sentences using `.predict_proba()`  
  * Extracts probability of projection (`pp_score`)  
  * Creates binary prediction: `proj_pred = 1` if `pp_score ≥ 0.5` else `0`  

* **Saves sentence-level results**  
  * Writes normative-only DataFrame with scores and predictions to `4_norm_sentences_pp_SUP.csv`  

* **Aggregates by organisation–year**  
  * Groups normative sentences by `(org, year)`  
  * Computes:  
    – `pp_mean`: mean projection probability  
    – `share_proj`: share of projection predictions  
    – `n_sent`: number of normative sentences  
  * Derives `share_prot = 1 − share_proj`  
  * Saves tidy aggregated DataFrame to `4_org_year_pp_SUP.csv`  


In [ ]:


import pandas as pd
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import joblib

BASE        = Path("/Users/bornaardehi/Desktop/TAD")
CORPUS      = BASE / "3_pred_documents.csv"   # from SVM step
GOLDSEEDS   = BASE / "4_PPGS.csv"             # validated PP gold seeds
MODEL_DIR   = BASE / "models"
MODEL_DIR.mkdir(exist_ok=True)

OUT_SENT    = BASE / "4_norm_sentences_pp_SUP.csv"
OUT_YEAR    = BASE / "4_org_year_pp_SUP.csv"

seeds = pd.read_csv(GOLDSEEDS)

# Encode label: proj -> 1, prot -> 0
seeds["y"] = seeds["label"].map({"proj": 1, "prot": 0})

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.9,
    lowercase=True
)

clf = LogisticRegression(
    solver="liblinear",
    C=1.0,
    class_weight="balanced",
    random_state=42
)

pipe = Pipeline([
    ("tfidf", vectorizer),
    ("logit", clf)
])

# Cross-validation for sanity
scores = cross_val_score(pipe, seeds["sentence"], seeds["y"], cv=5, scoring="accuracy")

# Fit final model
pipe.fit(seeds["sentence"], seeds["y"])
joblib.dump(pipe, MODEL_DIR / "pp_scaler.joblib")

df = pd.read_csv(CORPUS)
if "pred_normative" not in df.columns:
    raise ValueError("Corpus must have 'pred_normative' column from SVM step.")

norm_df = df[df["pred_normative"] == 1].copy()

pp_probs = pipe.predict_proba(norm_df["sentence"])[:, 1]  # probability of proj=1
norm_df["pp_score"] = pp_probs
norm_df["proj_pred"] = (norm_df["pp_score"] >= 0.5).astype(int)

norm_df.to_csv(OUT_SENT, index=False)

if {"org", "year"}.issubset(norm_df.columns):
    # make sure year is numeric (optional but handy)
    try:
        norm_df["year"] = norm_df["year"].astype(int)
    except Exception:
        pass

    agg = (norm_df.groupby(["org", "year"], as_index=False)
           .agg(pp_mean=("pp_score", "mean"),
                share_proj=("proj_pred", "mean"),
                n_sent=("sentence", "count")))

    # derive protection share from projection share
    agg["share_prot"] = 1.0 - agg["share_proj"]

    # tidy column order
    agg = agg[["org", "year", "pp_mean", "share_proj", "share_prot", "n_sent"]]

    agg.to_csv(OUT_YEAR, index=False)

### **Phase 4.3 — Diagnostics and optional retraining for projection/protection scaling**

This phase provides a diagnostic and retraining framework for the projection/protection (PP) classifier.  
It loads the gold seed set and normative-only sentences, evaluates cross-validation performance, displays top discriminative features, optionally retrains the model with custom vectorisation parameters, and applies the updated model to score projection/protection probabilities at both **sentence** and **organisation–year** levels.

**Process**

* **Imports required tools**  
  * `argparse` — for CLI argument parsing  
  * `os`, `sys` — for path handling and error exits  
  * `pandas`, `numpy` — for data handling  
  * `TfidfVectorizer` — for text feature extraction  
  * `LogisticRegression` — for binary classification  
  * `Pipeline` — for chaining TF–IDF and classifier  
  * `StratifiedKFold`, `cross_val_score`, `cross_val_predict` — for model evaluation  
  * `classification_report`, `accuracy_score`, `f1_score` — for performance metrics  
  * `Bunch` from `sklearn.utils` — for structured data returns  
  * `dump` from `joblib` — for saving trained models  

* **Helper functions**  
  * `find_text_col()` — identifies the text column in a DataFrame (prefers `sentence`, `text`, or `content`)  
  * `load_seeds(path)` — loads gold seed CSV, maps projection/protection labels to binary (`proj=1`, `prot=0`), returns features and labels  
  * `load_normative_sentences(path)` — loads corpus, filters to normative sentences based on `pred_normative` flag, returns text and metadata  
  * `make_pipeline()` — creates a TF–IDF + Logistic Regression pipeline with configurable n-gram range, min/max DF, and `C` parameter  
  * `show_top_features()` — prints the top k projection and protection n-grams with coefficients  
  * `evaluate_cv()` — runs stratified k-fold CV, prints accuracy/F1, and classification report  
  * `score_normative()` — applies model to normative sentences, outputs top projection and protection examples, and returns scored DataFrame  
  * `aggregate_org_year()` — aggregates projection/protection scores by organisation and year if metadata available  

* **Main workflow**  
  * Loads gold seed set (`--seeds_csv`), reports counts by class  
  * Builds TF–IDF + Logistic Regression pipeline with parameters from CLI arguments  
  * Evaluates model via cross-validation on seed set, printing accuracy, F1, and classification report  
  * Fits the model on the full seed set and prints top features  
  * Saves trained model to `--model_dir/pp_scaler.joblib`  

* **Scoring and output**  
  * Loads normative sentences from `--normative_csv`  
  * Scores each sentence with projection probability (`pp_score`) and binary projection label (`proj_pred`)  
  * Saves sentence-level results to `--sent_out`  
  * Aggregates by `(org, year)` to compute:  
    – `pp_mean`: mean projection probability  
    – `share_proj`: share of projection predictions  
    – `share_prot`: complement of `share_proj`  
    – `n_sent`: number of normative sentences in group  
  * Saves results to `--agg_out` if metadata available  


In [ ]:
import argparse
import os
import sys
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.utils import Bunch
from joblib import dump

# Helpers
def find_text_col(df, preferred=("sentence","text","content")):
    for c in preferred:
        if c in df.columns:
            return c
    # fallback: choose first object dtype column
    for c in df.columns:
        if df[c].dtype == "object":
            return c
    raise ValueError("Couldn't find a text column. Expected one of: " + ", ".join(preferred))

def load_seeds(path):
    df = pd.read_csv(path)
    text_col = find_text_col(df)
    # normalise label column
    if "label" in df.columns:
        y = df["label"]
    elif "class" in df.columns:
        y = df["class"]
    elif "pp_label" in df.columns:
        y = df["pp_label"]
    else:
        raise ValueError("Seed CSV needs a label column (label/class/pp_label).")
    y = y.str.strip().str.lower().map({"proj":1, "protection":0, "prot":0, "projection":1})
    if y.isna().any():
        bad = df.loc[y.isna()]
        raise ValueError(f"Unrecognised labels in seeds:\n{bad[['label'] if 'label' in df.columns else y.name].head()}")
    X = df[text_col].astype(str)
    return Bunch(X=X, y=y.values, text_col=text_col, raw=df)

def load_normative_sentences(path):
    df = pd.read_csv(path)
    # expect pred_normative column
    pred_col = None
    for c in ("pred_normative", "is_normative", "normative_pred", "normative"):
        if c in df.columns:
            pred_col = c
            break
    if pred_col is None:
        raise ValueError("3_pred_documents.csv must contain a normative flag column (pred_normative).")
    text_col = find_text_col(df)
    meta_cols = [c for c in ("org","year") if c in df.columns]
    df = df.loc[df[pred_col] == 1].copy()
    df[text_col] = df[text_col].astype(str)
    return Bunch(df=df, text_col=text_col, meta_cols=meta_cols)

def make_pipeline(ngram_lo, ngram_hi, min_df, max_df, C, use_class_weight=True):
    vec = TfidfVectorizer(
        ngram_range=(ngram_lo, ngram_hi),
        min_df=min_df,
        max_df=max_df,
        strip_accents="unicode",
        lowercase=True
    )
    lr = LogisticRegression(
        solver="liblinear",
        C=C,
        class_weight="balanced" if use_class_weight else None,
        max_iter=2000
    )
    return Pipeline([("tfidf", vec), ("lr", lr)])

def show_top_features(model, k=30):
    vec = model.named_steps["tfidf"]
    lr = model.named_steps["lr"]
    feats = np.array(vec.get_feature_names_out())
    coefs = lr.coef_.ravel()
    top_proj_idx = np.argsort(coefs)[-k:][::-1]
    top_prot_idx = np.argsort(coefs)[:k]
    print("\n=== Top projection n-grams ===")
    for i in top_proj_idx:
        print(f"{feats[i]:40s}  coef= {coefs[i]: .3f}")
    print("\n=== Top protection n-grams ===")
    for i in top_prot_idx:
        print(f"{feats[i]:40s}  coef= {coefs[i]: .3f}")

def evaluate_cv(pipe, X, y, n_splits=5, seed=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    acc = cross_val_score(pipe, X, y, cv=skf, scoring="accuracy")
    f1  = cross_val_score(pipe, X, y, cv=skf, scoring="f1")
    print(f"CV accuracy: {acc.mean():.3f} ± {acc.std():.3f}")
    print(f"CV F1 (proj=1): {f1.mean():.3f} ± {f1.std():.3f}")

    # detailed report via cross_val_predict
    yhat = cross_val_predict(pipe, X, y, cv=skf, method="predict")
    print("\nClassification report (proj=1, prot=0):")
    print(classification_report(y, yhat, digits=3))

def score_normative(pipe, norm: Bunch, top_k=20):
    X = norm.df[norm.text_col].values
    proba = pipe.predict_proba(X)[:,1]
    out = norm.df.copy()
    out["pp_score"] = proba
    out["proj_pred"] = (out["pp_score"] > 0.5).astype(int)
    # Top/bottom samples
    top = out.sort_values("pp_score", ascending=False).head(top_k)
    bot = out.sort_values("pp_score", ascending=True).head(top_k)
    print("\n=== Top projection sentences ===")
    for i, row in top.iterrows():
        preview = row[norm.text_col].replace("\n"," ")[:280]
        print(f"[{row.get('org','?')}/{row.get('year','?')}]  {row['pp_score']:.3f}  {preview}")
    print("\n=== Top protection sentences ===")
    for i, row in bot.iterrows():
        preview = row[norm.text_col].replace("\n"," ")[:280]
        print(f"[{row.get('org','?')}/{row.get('year','?')}]  {row['pp_score']:.3f}  {preview}")
    return out

def aggregate_org_year(df, meta_cols):
    need = {"org","year"}
    have = set(meta_cols)
    if not need.issubset(have):
        print("(!) org/year not found — skipping org-year aggregation.")
        return None
    gp = df.groupby(["org","year"])
    agg = gp.agg(
        pp_mean=("pp_score","mean"),
        share_proj=("proj_pred","mean"),
        n_sent=("proj_pred","size")
    ).reset_index()
    agg["share_prot"] = 1.0 - agg["share_proj"]
    # order
    agg = agg[["org","year","pp_mean","share_proj","share_prot","n_sent"]]
    return agg


def main(args):
    seeds = load_seeds(args.seeds_csv)
    print(f"Loaded seeds: {len(seeds.X)} (proj={seeds.y.sum()}, prot={len(seeds.y)-seeds.y.sum()})")

    pipe = make_pipeline(
        ngram_lo=args.ngram_lo,
        ngram_hi=args.ngram_hi,
        min_df=args.min_df,
        max_df=args.max_df,
        C=args.C,
        use_class_weight=not args.no_class_weight
    )

    print("\n[Step 5] Cross-validation on seeds")
    evaluate_cv(pipe, seeds.X, seeds.y, n_splits=args.cv_folds)

    print("\nFit on full seed set…")
    pipe.fit(seeds.X, seeds.y)
    show_top_features(pipe, k=args.top_k)

    # Save model
    os.makedirs(args.model_dir, exist_ok=True)
    model_path = os.path.join(args.model_dir, "pp_scaler.joblib")
    dump(pipe, model_path)
    print(f"\n✓ Saved model to {model_path}")

    # Load normative sentences and score
    print("\nLoad normative sentences and re-score…")
    norm = load_normative_sentences(args.normative_csv)
    scored = score_normative(pipe, norm, top_k=args.top_examples)

    # Persist sentence-level scores
    sent_out = args.sent_out
    scored.to_csv(sent_out, index=False)
    print(f"✓ Saved sentence-level scores to {sent_out}")

    # Org-year aggregate (if meta available)
    agg = aggregate_org_year(scored, norm.meta_cols)
    if agg is not None:
        agg_out = args.agg_out
        agg.to_csv(agg_out, index=False)
        print(f"✓ Saved org-year aggregates to {agg_out}")

if __name__ == "__main__":
    p = argparse.ArgumentParser(description="Diagnostics and retrain for projection/protection scaling")
    p.add_argument("--seeds_csv", default="4_PPGS.csv", help="Gold seed file")
    p.add_argument("--normative_csv", default="3_pred_documents.csv", help="Corpus with pred_normative flag")

    # Vectoriser / model knobs
    p.add_argument("--ngram_lo", type=int, default=1)
    p.add_argument("--ngram_hi", type=int, default=3)
    p.add_argument("--min_df", type=int, default=3)
    p.add_argument("--max_df", type=float, default=0.85)
    p.add_argument("--C", type=float, default=1.0)
    p.add_argument("--no_class_weight", action="store_true", help="Disable class_weight=balanced")

    # CV / reporting
    p.add_argument("--cv_folds", type=int, default=5)
    p.add_argument("--top_k", type=int, default=30, help="How many top features to print per side")
    p.add_argument("--top_examples", type=int, default=20, help="How many top/bottom sentences to print")

    # Paths out
    p.add_argument("--model_dir", default="models")
    p.add_argument("--sent_out", default="4_norm_sentences_pp_SUP.csv")
    p.add_argument("--agg_out", default="4_org_year_pp_SUP.csv")

    args = p.parse_args()
    main(args)

### **Phase 5.0 — Descriptive statistical analysis of projection/protection results**

This phase produces descriptive statistics, tests of group differences, temporal correlation analyses, and volatility measures based on the organisation–year projection/protection dataset from Phase 4.2 or 4.3 (`4_org_year_pp_SUP.csv`).  
The outputs are saved as HTML tables for direct inspection and reporting.

**Process**

* **Imports required tools**  
  * `pandas` (`pip install pandas`) — for data manipulation and HTML export  
  * `numpy` (`pip install numpy`) — for numerical operations  
  * `f_oneway` from `scipy.stats` — for one-way ANOVA  
  * `pearsonr` from `scipy.stats` — for Pearson correlation  
  * `pairwise_tukeyhsd` from `statsmodels.stats.multicomp` — for Tukey post-hoc tests  

* **Loads and prepares dataset**  
  * Reads `4_org_year_pp_SUP.csv` containing organisation–year projection/protection shares  
  * Converts `year` column to integer type  

* **Descriptive statistics**  
  * **Overall stats** — computes count, mean, std, min, 25%, 50%, 75%, max for `share_proj`, `share_prot`, and `pp_mean`  
  * Adds interquartile range (IQR)  
  * Saves results to `desc_stats_overall.html`  
  * **By-organisation stats** — computes mean, std, min, max per organisation for the same variables  
  * Saves results to `desc_stats_by_org.html`  

* **ANOVA and Tukey post-hoc tests**  
  * Performs one-way ANOVA on `share_proj` grouped by organisation  
  * Saves F-statistic and p-value to `anova_results.html`  
  * Runs Tukey HSD to identify significant differences between organisations  
  * Saves results to `tukey_results.html`  

* **Time-trend correlations**  
  * For each organisation with >1 unique year, computes Pearson correlation (`r`) between `year` and `share_proj` (with p-value)  
  * Saves results to `trend_correlation_by_org.html`  

* **Volatility analysis**  
  * Sorts dataset by `org` and `year`  
  * Computes year-to-year change (`delta_proj`) in `share_proj` within each organisation  
  * Aggregates volatility stats per organisation (mean, std, max, min)  
  * Saves results to `volatility_by_org.html`  
  * Identifies top 20 largest absolute changes across all organisation–year pairs  
  * Saves results to `top_volatility_cases.html`  

* **Output**  
  * All descriptive and inferential results are exported as separate HTML tables for direct viewing  
  * Prints a completion message confirming successful saves  


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, pearsonr
from statsmodels.stats.multicomp import pairwise_tukeyhsd

df = pd.read_csv("4_org_year_pp_SUP.csv")
df["year"] = df["year"].astype(int)

# Descriptive Stats
overall_stats = df[["share_proj", "share_prot", "pp_mean"]].describe().T
overall_stats["iqr"] = df[["share_proj", "share_prot", "pp_mean"]].quantile(0.75) - \
                       df[["share_proj", "share_prot", "pp_mean"]].quantile(0.25)
overall_stats.to_html("desc_stats_overall.html")

by_org_stats = df.groupby("org")[["share_proj", "share_prot", "pp_mean"]].agg(
    ["mean", "std", "min", "max"]
)
by_org_stats.to_html("desc_stats_by_org.html")

# ANOVA + Tukey
groups = [g["share_proj"].values for _, g in df.groupby("org")]
anova_stat, anova_p = f_oneway(*groups)
pd.DataFrame({"F-stat": [anova_stat], "p-value": [anova_p]}).to_html("anova_results.html")

tukey = pairwise_tukeyhsd(endog=df["share_proj"], groups=df["org"], alpha=0.05)
tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
tukey_df.to_html("tukey_results.html")

# Time-trend correlation
trend_corrs = []
for org, g in df.groupby("org"):
    if g["year"].nunique() > 1:
        r, p = pearsonr(g["year"], g["share_proj"])
        trend_corrs.append({"org": org, "pearson_r": r, "p_value": p})

trend_df = pd.DataFrame(trend_corrs)
trend_df.to_html("trend_correlation_by_org.html")

# Volatility
df_sorted = df.sort_values(["org", "year"])
df_sorted["delta_proj"] = df_sorted.groupby("org")["share_proj"].diff()

volatility_stats = df_sorted.groupby("org")["delta_proj"].agg(["mean", "std", "max", "min"])
volatility_stats.to_html("volatility_by_org.html")

top_changes = df_sorted.sort_values("delta_proj", key=abs, ascending=False).head(20)
top_changes[["org", "year", "delta_proj"]].to_html("top_volatility_cases.html")


### **Phase 5.1 –– Graphs and visualizations of projection vs protection**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


BASE = Path("/Users/bornaardehi/Desktop/TAD")
IN = BASE / "4_org_year_pp_SUP.csv"
OUTS = {
    "proj_over_time_png": BASE / "viz_proj_share_over_time.png",
    "proj_over_time_svg": BASE / "viz_proj_share_over_time.svg",
    "prot_over_time_png": BASE / "viz_prot_share_over_time.png",
    "prot_over_time_svg": BASE / "viz_prot_share_over_time.svg",
    "avg_by_org_png":     BASE / "viz_proj_vs_prot_by_org.png",
    "avg_by_org_svg":     BASE / "viz_proj_vs_prot_by_org.svg",
    "extremes_csv":       BASE / "extreme_years_proj.csv",
    "extremes_prot_csv":  BASE / "extreme_years_prot.csv",
    "facet_proj_png":     BASE / "viz_proj_share_facets.png",
    "facet_proj_svg":     BASE / "viz_proj_share_facets.svg",
}

BASE.mkdir(parents=True, exist_ok=True)

# Style
sns.set_theme(context="talk", style="whitegrid")
palette = sns.color_palette("colorblind")

df = pd.read_csv(IN)
keep = ["org", "year", "share_proj", "share_prot", "pp_mean"]
df = df[[c for c in keep if c in df.columns]].copy()

# types & dedupe
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
df = df.dropna(subset=["org", "year", "share_proj", "share_prot"]).drop_duplicates()
# sort for nicer lines
df = df.sort_values(["org", "year"])

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="year", y="share_proj", hue="org", marker="o", palette=palette, linewidth=2)
plt.title("Projection Share Over Time by Organisation", fontsize=14)
plt.ylabel("Share (Projection)")
plt.xlabel("Year")
# Trim legend if many orgs
handles, labels = plt.gca().get_legend_handles_labels()
if len(labels) > 20:
    plt.legend([], [], frameon=False)
else:
    plt.legend(title="Organisation", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.savefig(OUTS["proj_over_time_png"], dpi=300)
plt.savefig(OUTS["proj_over_time_svg"])
plt.close()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x="year", y="share_prot", hue="org", marker="o", palette=palette, linewidth=2)
plt.title("Protection Share Over Time by Organisation", fontsize=14)
plt.ylabel("Share (Protection)")
plt.xlabel("Year")
handles, labels = plt.gca().get_legend_handles_labels()
if len(labels) > 20:
    plt.legend([], [], frameon=False)
else:
    plt.legend(title="Organisation", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.savefig(OUTS["prot_over_time_png"], dpi=300)
plt.savefig(OUTS["prot_over_time_svg"])
plt.close()

if df["org"].nunique() > 8:
    g = sns.FacetGrid(df, col="org", col_wrap=4, height=2.8, sharey=True)
    g.map_dataframe(sns.lineplot, x="year", y="share_proj", marker="o", linewidth=1.8)
    g.set_titles(col_template="{col_name}")
    g.set_axis_labels("Year", "Share (Projection)")
    plt.tight_layout()
    plt.savefig(OUTS["facet_proj_png"], dpi=300)
    plt.savefig(OUTS["facet_proj_svg"])
    plt.close()

avg_df = (
    df.groupby("org", as_index=False)
      .agg(share_proj=("share_proj", "mean"), share_prot=("share_prot", "mean"))
      .sort_values("share_proj", ascending=False)
)

plt.figure(figsize=(12, 6))
ax = avg_df.plot(
    x="org",
    y=["share_proj", "share_prot"],
    kind="bar",
    figsize=(12, 6),
    legend=True
)
plt.title("Average Projection vs Protection by Organisation", fontsize=14)
plt.ylabel("Average Share")
plt.xlabel("")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(OUTS["avg_by_org_png"], dpi=300)
plt.savefig(OUTS["avg_by_org_svg"])
plt.close()

def extremes_by(group, col, n=2):
    # Copy to avoid chained assignment, and handle cases with < n rows gracefully
    top = group.nlargest(min(n, len(group)), columns=col)[["year", col]].copy()
    bot = group.nsmallest(min(n, len(group)), columns=col)[["year", col]].copy()
    top["which"] = "top"
    bot["which"] = "bottom"
    return pd.concat([top, bot], ignore_index=True)

ext_list = []
for org, g in df.groupby("org"):
    ex = extremes_by(g, "share_proj", n=2)
    ex.insert(0, "org", org)
    ext_list.append(ex)
ext_proj = pd.concat(ext_list, ignore_index=True).sort_values(["org", "which", "year"])
ext_proj.rename(columns={"share_proj": "value"}, inplace=True)
ext_proj.to_csv(OUTS["extremes_csv"], index=False)

ext_list = []
for org, g in df.groupby("org"):
    ex = extremes_by(g, "share_prot", n=2)
    ex.insert(0, "org", org)
    ext_list.append(ex)
ext_prot = pd.concat(ext_list, ignore_index=True).sort_values(["org", "which", "year"])
ext_prot.rename(columns={"share_prot": "value"}, inplace=True)
ext_prot.to_csv(OUTS["extremes_prot_csv"], index=False)

for k, p in OUTS.items():

### **Phase 5.1 –– Graphs and visualizations of normative vs substansive**

Since this step uses a different dataset but is following 'stage 1' of our research, it is not an issue to execute it after the previous step.

In [ ]:

import os
import math
import itertools
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import multitest
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter, MaxNLocator
from datetime import datetime
import html

DATA_PATH = "3_pred_documents.csv"     
OUTPUT_DIR = "norm_func_outputs"       
MIN_COUNT_FOR_ORG_IN_CHISQ = 30        # rare orgs for chi-square
TOP_N_FOR_FISHER = 12                  # pairwise Fisher across top-N orgs by volume
TOP_N_FOR_BAR = 15                     # top-N orgs in bar plot
HEATMAP_TOP_K = 20                     # top-K orgs in heatmap
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Stargazer style of output

_STARGAZER_CSS = """
<style>
/* Classic Stargazer-like table with box rules */
.stgz { border-collapse: collapse; margin: 1.2rem 0; font-size: 14px; font-family: 'Times New Roman', Times, serif; }
.stgz thead tr.rule-top    td, .stgz thead tr.rule-top    th { border-top: 2px solid #000; }
.stgz tfoot tr.rule-bottom td, .stgz tfoot tr.rule-bottom th { border-bottom: 2px solid #000; }
.stgz td, .stgz th { padding: 6px 10px; }
.stgz .center { text-align: center; }
.stgz .right  { text-align: right;  }
.stgz .left   { text-align: left;   }
.stgz .depvar { text-align: center; font-style: italic; }
.stgz .note   { font-size: 12px; text-align: right; padding-top: 6px; }
.stgz .hr { border-bottom: 1px solid #000; height: 0; }
</style>
"""

def _stars(p):
    if p < 0.001: return "***"
    if p < 0.01:  return "**"
    if p < 0.05:  return "*"
    return ""

def _pseudo_r2(results):
    # McFadden pseudo R^2 ~ 1 - (deviance/null_deviance)
    try:
        return max(0.0, 1.0 - (results.deviance / results.null_deviance))
    except Exception:
        try:
            # fallback if available
            return max(0.0, 1.0 - (results.llf / results.llnull))
        except Exception:
            return np.nan

def _escape(x): return html.escape(str(x))

def render_stargazer(models, model_names, depvar_label, file_path, title=None):
    """
    models: list of statsmodels Results (GLMResults)
    model_names: list like ["(M1)","(M2)",...]
    depvar_label: e.g., "Dependent variable: pred_normative"
    file_path: output HTML path
    """
    # Collect full set of terms across models
    all_terms = set()
    for m in models:
        all_terms |= set(m.params.index.tolist())
    # Stargazer usually hides intercept text label; we keep "Intercept" row last
    terms = [t for t in sorted(all_terms) if t != "Intercept"] + (["Intercept"] if "Intercept" in all_terms else [])

    # Build header rows
    k = len(models)
    header = []
    header.append(f"<tr class='rule-top'><th colspan='{k+1}'></th></tr>")
    if title:
        header.append(f"<tr><th colspan='{k+1}' class='center'>{_escape(title)}</th></tr>")
    header.append(f"<tr><th colspan='{k+1}' class='depvar'>{_escape(depvar_label)}</th></tr>")
    # Model labels row
    labels = "".join([f"<th class='center'>{_escape(mn)}</th>" for mn in model_names])
    header.append(f"<tr><th></th>{labels}</tr>")
    # small horizontal rule
    header.append(f"<tr><th colspan='{k+1}' class='hr'></th></tr>")

    # Body rows: for each term, show coef+stars, then SE row in parens
    body = []
    for term in terms:
        row_coef = [f"<td class='left'>{_escape(term)}</td>"]
        row_se   = ["<td></td>"]
        for m in models:
            if term in m.params.index:
                coef = m.params[term]
                se   = m.bse[term]
                p    = m.pvalues[term]
                row_coef.append(f"<td class='center'>{coef:.3f}{_stars(p)}</td>")
                row_se.append(f"<td class='center'>({_escape(f'{se:.3f}')})</td>")
            else:
                row_coef.append("<td class='center'></td>")
                row_se.append("<td class='center'></td>")
        body.append("<tr>" + "".join(row_coef) + "</tr>")
        body.append("<tr>" + "".join(row_se)   + "</tr>")

    # Footer: Observations, Pseudo R^2
    n_row = ["<td class='left'>Observations</td>"] + [f"<td class='center'>{int(getattr(m,'nobs',np.nan))}</td>" for m in models]
    r2_row = ["<td class='left'>Pseudo R<sup>2</sup></td>"] + [f"<td class='center'>{_pseudo_r2(m):.3f}</td>" for m in models]

    footer = []
    footer.append("<tr><td colspan='999' class='hr'></td></tr>")
    footer.append("<tr>" + "".join(n_row) + "</tr>")
    footer.append("<tr>" + "".join(r2_row) + "</tr>")
    footer.append("<tr class='rule-bottom'><td colspan='999'></td></tr>")
    footer.append("<tr><td colspan='999' class='note'>* p&lt;0.05; ** p&lt;0.01; *** p&lt;0.001</td></tr>")

    table = "<table class='stgz'>" + "".join(header + body + footer) + "</table>"
    html_doc = f"<!DOCTYPE html><html><head><meta charset='utf-8'>{_STARGAZER_CSS}</head><body>{table}</body></html>"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(html_doc)

# helpers

def cramers_v_from_chi2(chi2, n, r, k, bias_correct=True):
    phi2 = chi2 / n
    if bias_correct:
        phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
        rcorr = r - ((r-1)**2)/(n-1)
        kcorr = k - ((k-1)**2)/(n-1)
        denom = min((kcorr-1), (rcorr-1))
        return np.sqrt(phi2corr / denom) if denom > 0 else np.nan
    else:
        denom = min(k-1, r-1)
        return np.sqrt(phi2 / denom) if denom > 0 else np.nan

def odds_ratio_ci(table_2x2, alpha=0.05):
    a,b = table_2x2[0]
    c,d = table_2x2[1]
    if 0 in [a,b,c,d]:
        a,b,c,d = a+0.5, b+0.5, c+0.5, d+0.5
    or_est = (a*d)/(b*c)
    se = math.sqrt(1/a + 1/b + 1/c + 1/d)
    z = stats.norm.ppf(1 - alpha/2)
    lo, hi = math.exp(math.log(or_est) - z*se), math.exp(math.log(or_est) + z*se)
    return or_est, lo, hi

def add_or_table(results):
    coefs = results.params.copy()
    conf = results.conf_int()
    out = pd.DataFrame({
        "coef": coefs,
        "OR": np.exp(coefs),
        "CI_low": np.exp(conf[0]),
        "CI_high": np.exp(conf[1]),
        "p": results.pvalues
    })
    return out

def make_decade(year):
    try:
        y = int(year)
        return f"{(y // 10) * 10}s"
    except Exception:
        return np.nan

# load data

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Could not find file at: {DATA_PATH}")

df = pd.read_csv(DATA_PATH)

expected_cols = {"doc_id","org","year","sent_id","sentence","pred_normative"}
missing = expected_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df["pred_normative"] = pd.to_numeric(df["pred_normative"], errors="coerce").fillna(0).astype(int)
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df = df.dropna(subset=["year", "org", "doc_id"])
df["year"] = df["year"].astype(int)
df["org"] = df["org"].astype(str).str.strip()
df["doc_id"] = df["doc_id"].astype(str)
df["decade"] = df["year"].apply(make_decade)
df["pred_normative"] = df["pred_normative"].clip(0,1)

# Descriptive stats

overall_rate = df["pred_normative"].mean()

by_org = (df.groupby("org")["pred_normative"]
            .agg(n="count", norm_rate="mean")
            .sort_values("n", ascending=False))
by_org["norm_rate_pct"] = 100*by_org["norm_rate"]

by_year = (df.groupby("year")["pred_normative"]
             .agg(n="count", norm_rate="mean"))
by_year["norm_rate_pct"] = 100*by_year["norm_rate"]

# Save descriptive tables (same folder, simple HTML; these aren’t regression tables)
def save_simple_table(df_in, path, title):
    # light wrapper to keep style consistent with SG framing
    cols = "".join([f"<th>{_escape(c)}</th>" for c in ([''] + df_in.columns.tolist())])
    head = f"<tr class='rule-top'><th colspan='{len(df_in.columns)+1}'></th></tr>"
    head += f"<tr><th colspan='{len(df_in.columns)+1}' class='center'>{_escape(title)}</th></tr>"
    head += f"<tr><th></th>{cols[4:]}</tr>"
    rows = []
    for idx, row in df_in.iterrows():
        rows.append("<tr>" + f"<td class='left'>{_escape(idx)}</td>" +
                    "".join([f"<td class='right'>{_escape(v)}</td>" for v in row]) + "</tr>")
    foot = "<tr class='rule-bottom'><td colspan='999'></td></tr>"
    html_doc = f"<!DOCTYPE html><html><head><meta charset='utf-8'>{_STARGAZER_CSS}</head><body><table class='stgz'>{head}{''.join(rows)}{foot}</table></body></html>"
    with open(path, "w", encoding="utf-8") as f:
        f.write(html_doc)

overview = pd.DataFrame.from_dict(
    {
        "Total sentences": [len(df)],
        "Total documents": [df["doc_id"].nunique()],
        "Total organizations": [df["org"].nunique()],
        "Year min": [int(df["year"].min())],
        "Year max": [int(df["year"].max())],
        "Overall normative share (%)": [round(overall_rate*100.0, 2)]
    }, orient="index", columns=["Value"]
)
save_simple_table(overview, os.path.join(OUTPUT_DIR, "01_overview.html"), "Overview")

tmp2 = by_org.copy()
tmp2["normative_share_%"] = tmp2["norm_rate_pct"].round(1)
tmp2 = tmp2.drop(columns=["norm_rate","norm_rate_pct"])
save_simple_table(tmp2, os.path.join(OUTPUT_DIR, "02_by_organization.html"), "Normative vs Functional by Organization")

tmpy2 = by_year.copy()
tmpy2["normative_share_%"] = tmpy2["norm_rate_pct"].round(1)
tmpy2 = tmpy2.drop(columns=["norm_rate","norm_rate_pct"])
save_simple_table(tmpy2, os.path.join(OUTPUT_DIR, "03_by_year.html"), "Normative vs Functional by Year")

# Chi-sqr test

org_counts = df["org"].value_counts()
keep_orgs = org_counts[org_counts >= MIN_COUNT_FOR_ORG_IN_CHISQ].index
df_chi = df.copy()
df_chi["org_pooled"] = np.where(df_chi["org"].isin(keep_orgs), df_chi["org"], "Other")

cont = pd.crosstab(df_chi["pred_normative"], df_chi["org_pooled"])

if cont.size > 0 and cont.shape[1] >= 2:
    chi2, p, dof, expected = stats.chi2_contingency(cont)
    n = cont.to_numpy().sum()
    cv = cramers_v_from_chi2(chi2, n, cont.shape[0], cont.shape[1], bias_correct=True)
    chi_summary = pd.DataFrame.from_dict(
        {"chi2":[chi2], "dof":[dof], "p_value":[p], "Cramér_V":[cv], "N":[int(n)],
         "cells exp<5":[int((expected<5).sum())]}, orient="columns")
    save_simple_table(chi_summary.set_index(pd.Index([""])), os.path.join(OUTPUT_DIR, "04a_chi_square_summary.html"),
                      "Chi-square: normative × org_pooled")
    save_simple_table(cont, os.path.join(OUTPUT_DIR, "04b_chi_square_contingency.html"),
                      "Contingency (rows=pred_normative, cols=org_pooled)")
    exp_df = pd.DataFrame(expected, index=cont.index, columns=cont.columns)
    save_simple_table(exp_df, os.path.join(OUTPUT_DIR, "04c_chi_square_expected.html"),
                      "Expected Counts")
else:
    save_simple_table(pd.DataFrame({"note":["Chi-square skipped (insufficient variation)."]}).set_index(pd.Index([""])),
                      os.path.join(OUTPUT_DIR, "04a_chi_square_summary.html"),
                      "Chi-square: skipped")

# Pairwise Fisher (top-N orgs)
top_orgs = org_counts.index[:TOP_N_FOR_FISHER]
pairs = list(itertools.combinations(top_orgs, 2))
fisher_rows = []
for a,b in pairs:
    sub = df[df["org"].isin([a,b])]
    tab = pd.crosstab(sub["org"], sub["pred_normative"]).reindex([a,b], fill_value=0)
    if tab.shape[0] != 2:
        continue
    a_norm = tab.loc[a, 1] if 1 in tab.columns else 0
    a_non  = tab.loc[a, 0] if 0 in tab.columns else 0
    b_norm = tab.loc[b, 1] if 1 in tab.columns else 0
    b_non  = tab.loc[b, 0] if 0 in tab.columns else 0
    if (a_norm + a_non == 0) or (b_norm + b_non == 0):
        continue
    table = [[a_norm, a_non],[b_norm, b_non]]
    oddsratio, pval = stats.fisher_exact(table, alternative='two-sided')
    or_est, lo, hi = odds_ratio_ci(table)
    fisher_rows.append({
        "org_A": a, "org_B": b,
        "A_norm_rate_%": (a_norm / max(1, (a_norm + a_non))) * 100.0,
        "B_norm_rate_%": (b_norm / max(1, (b_norm + b_non))) * 100.0,
        "OR_A_vs_B": or_est,
        "CI_low": lo,
        "CI_high": hi,
        "p_two_sided": pval,
        "N": a_norm + a_non + b_norm + b_non
    })

fisher_df = pd.DataFrame(fisher_rows)
if not fisher_df.empty:
    fisher_df["p_fdr"] = multitest.multipletests(fisher_df["p_two_sided"], method="fdr_bh")[1]
    fisher_df = fisher_df.sort_values("p_fdr").reset_index(drop=True)
    save_simple_table(fisher_df, os.path.join(OUTPUT_DIR, "05_fisher_pairwise.html"),
                      "Pairwise Fisher (Top-N orgs), FDR-corrected")
else:
    save_simple_table(pd.DataFrame({"note":["Pairwise Fisher: no valid org pairs to test."]}).set_index(pd.Index([""])),
                      os.path.join(OUTPUT_DIR, "05_fisher_pairwise.html"),
                      "Pairwise Fisher: skipped")

# decade chi-square
if df["decade"].notna().any():
    cont_dec = pd.crosstab(df["pred_normative"], df["decade"])
    if cont_dec.size > 0 and cont_dec.shape[1] >= 2:
        chi2d, pd_, dofd, expd = stats.chi2_contingency(cont_dec)
        ndec = cont_dec.to_numpy().sum()
        cvd = cramers_v_from_chi2(chi2d, ndec, cont_dec.shape[0], cont_dec.shape[1], bias_correct=True)
        chi_dec = pd.DataFrame.from_dict(
            {"chi2":[chi2d], "dof":[dofd], "p_value":[pd_], "Cramér_V":[cvd], "N":[int(ndec)]}, orient="columns")
        save_simple_table(chi_dec.set_index(pd.Index([""])),
                          os.path.join(OUTPUT_DIR, "06_decade_chi_square_summary.html"),
                          "Chi-square: normative × decade")
        save_simple_table(cont_dec, os.path.join(OUTPUT_DIR, "06_decade_contingency.html"),
                          "Contingency (rows=pred_normative, cols=decade)")
        expd_df = pd.DataFrame(expd, index=cont_dec.index, columns=cont_dec.columns)
        save_simple_table(expd_df, os.path.join(OUTPUT_DIR, "06_decade_expected.html"),
                          "Expected Counts (decade)")
    else:
        save_simple_table(pd.DataFrame({"note":["Decade chi-square skipped: insufficient variation."]}).set_index(pd.Index([""])),
                          os.path.join(OUTPUT_DIR, "06_decade_chi_square_summary.html"),
                          "Chi-square (decade): skipped")

# Log regression

glm_m1 = smf.glm("pred_normative ~ year",
                 data=df,
                 family=sm.families.Binomial())
m1 = glm_m1.fit(cov_type="cluster", cov_kwds={"groups": df["doc_id"]})

glm_m2 = smf.glm("pred_normative ~ year + C(org)",
                 data=df,
                 family=sm.families.Binomial())
m2 = glm_m2.fit(cov_type="cluster", cov_kwds={"groups": df["doc_id"]})

render_stargazer(
    models=[m1, m2],
    model_names=["(M1)","(M2)"],
    depvar_label="Dependent variable: pred_normative",
    file_path=os.path.join(OUTPUT_DIR, "07_models_sentence_glm.html"),
    title=None
)

# OR table (nice-to-have, simple SG frame)
m2_tbl = (add_or_table(m2)
          .reset_index()
          .rename(columns={"index":"term"}))
m2_tbl["term"] = (m2_tbl["term"].str.replace("C(org)[T.", "org: ", regex=False)
                               .str.replace("]", "", regex=False))
m2_tbl = m2_tbl.sort_values("p")[["term","OR","CI_low","CI_high","p"]]
save_simple_table(m2_tbl.set_index("term"),
                  os.path.join(OUTPUT_DIR, "08b_org_effects_or.html"),
                  "Organization Effects (Odds Ratios, GLM with org FE)")

# Time series analysis

agg_year = df.groupby("year")["pred_normative"].agg(y_sum="sum", n="count").reset_index()
agg_year["rate"] = agg_year["y_sum"] / agg_year["n"]

if len(agg_year) >= 2:
    # Use var_weights to avoid cov_type warning
    glm1 = smf.glm("rate ~ year",
                   data=agg_year,
                   family=sm.families.Binomial(),
                   var_weights=agg_year["n"])
    ts1 = glm1.fit(cov_type="HAC", cov_kwds={"maxlags":1})
    render_stargazer(
        models=[ts1],
        model_names=["(M1)"],
        depvar_label="Dependent variable: rate (yearly normative share)",
        file_path=os.path.join(OUTPUT_DIR, "09_glm_yearly_trend.html"),
        title=None
    )
else:
    save_simple_table(pd.DataFrame({"note":["Year-level GLM skipped: < 2 years present."]}).set_index(pd.Index([""])),
                      os.path.join(OUTPUT_DIR, "09_glm_yearly_trend.html"),
                      "Year-level GLM: skipped")

panel = (df.groupby(["org","year"])["pred_normative"]
           .agg(y_sum="sum", n="count").reset_index())
panel["rate"] = panel["y_sum"]/panel["n"]

if not panel.empty and panel["org"].nunique() >= 2:
    glm2 = smf.glm("rate ~ year + C(org)",
                   data=panel,
                   family=sm.families.Binomial(),
                   var_weights=panel["n"])
    ts2 = glm2.fit(cov_type="cluster", cov_kwds={"groups": panel["org"]})
    render_stargazer(
        models=[ts2],
        model_names=["(M1)"],
        depvar_label="Dependent variable: rate (org-year share)",
        file_path=os.path.join(OUTPUT_DIR, "10_glm_org_year_panel.html"),
        title=None
    )
else:
    save_simple_table(pd.DataFrame({"note":["Org-year GLM skipped: insufficient org/year variation."]}).set_index(pd.Index([""])),
                      os.path.join(OUTPUT_DIR, "10_glm_org_year_panel.html"),
                      "Org-year GLM: skipped")

# Visualisation

plt.close("all")

# (A) Top orgs by volume with normative share
top_org_plot = by_org.head(TOP_N_FOR_BAR).copy()
if not top_org_plot.empty:
    fig, ax = plt.subplots(figsize=(10, 6))
    labels_rev = top_org_plot.index[::-1]
    vals_rev = np.nan_to_num(top_org_plot["norm_rate"].to_numpy()[::-1], nan=0.0)
    bars = ax.barh(labels_rev, vals_rev)
    ax.set_xlabel("Normative share")
    ax.set_xlim(0, 1)
    ax.xaxis.set_major_formatter(PercentFormatter(1.0))
    ax.set_title(f"Normative share by organization (Top {len(top_org_plot)} by volume)")
    for b, val in zip(bars, vals_rev):
        ax.text(min(val + 0.01, 0.98), b.get_y()+b.get_height()/2, f"{val*100:.1f}%", va="center")
    plt.tight_layout()
    fig.savefig(os.path.join(OUTPUT_DIR, "11_plot_bar_by_org.png"), dpi=200)
    plt.close(fig)

# (B) Time trend of normative rate (yearly)
if not agg_year.empty:
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(agg_year["year"], agg_year["rate"], marker="o", linewidth=1.5)
    ax.yaxis.set_major_formatter(PercentFormatter(1.0))
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_xlabel("Year")
    ax.set_ylabel("Normative share")
    ax.set_title("Yearly normative share over time")
    plt.tight_layout()
    fig.savefig(os.path.join(OUTPUT_DIR, "12_plot_yearly_trend.png"), dpi=200)
    plt.close(fig)

# (C) Heatmap: org (top K) x year normative rate
heat_orgs = df["org"].value_counts().head(HEATMAP_TOP_K).index
heat = (df[df["org"].isin(heat_orgs)]
        .groupby(["org","year"])["pred_normative"]
        .mean().unstack("year"))

if (heat.shape[0] > 0) and (heat.shape[1] > 0):
    fig, ax = plt.subplots(figsize=(12, max(6, 0.45*heat.shape[0])))
    im = ax.imshow(heat.values, aspect="auto", interpolation="nearest")
    im.set_clim(0, 1)
    ax.set_yticks(range(len(heat.index)))
    ax.set_yticklabels(heat.index)
    ax.set_xticks(range(len(heat.columns)))
    ax.set_xticklabels(heat.columns, rotation=90)
    ax.set_title(f"Normative share heatmap (Top {len(heat_orgs)} orgs by volume)")
    cbar = plt.colorbar(im, ax=ax, fraction=0.02, pad=0.02)
    cbar.ax.yaxis.set_major_formatter(PercentFormatter(1.0))
    plt.tight_layout()
    fig.savefig(os.path.join(OUTPUT_DIR, "13_plot_heatmap_org_year.png"), dpi=200)
    plt.close(fig)

# (D) Coefficient display: year effect + some org effects from Model 2
try:
    coef_df = add_or_table(m2).reset_index().rename(columns={"index":"term"})
    coef_df["is_org"] = coef_df["term"].str.startswith("C(org)")
    big_org_terms = ["C(org)[T." + o + "]" for o in df["org"].value_counts().index[:10] if "C(org)[T." + o + "]" in set(coef_df["term"])]
    plot_terms = (["year"] if "year" in coef_df["term"].values else []) + big_org_terms
    plot_subset = coef_df[coef_df["term"].isin(plot_terms)].copy()
    if not plot_subset.empty:
        if "year" in plot_subset["term"].values:
            plot_subset = pd.concat([
                plot_subset[plot_subset["term"]=="year"],
                plot_subset[(plot_subset["term"]!="year")].sort_values("OR", ascending=False)
            ])
        fig, ax = plt.subplots(figsize=(10, 6))
        ypos = np.arange(len(plot_subset))
        x = plot_subset["OR"].to_numpy()
        lo = plot_subset["CI_low"].to_numpy()
        hi = plot_subset["CI_high"].to_numpy()
        ax.errorbar(x, ypos, xerr=[x - lo, hi - x], fmt='o', capsize=3)
        ax.axvline(1.0, linewidth=1)
        ax.set_yticks(ypos)
        pretty_labels = plot_subset["term"].str.replace("C(org)[T.", "org: ", regex=False).str.replace("]", "", regex=False).tolist()
        ax.set_yticklabels(pretty_labels)
        ax.set_xlabel("Odds Ratio (GLM Binomial with org FE)")
        ax.set_title("Selected effects with 95% CIs")
        plt.tight_layout()
        fig.savefig(os.path.join(OUTPUT_DIR, "14_plot_coef.png"), dpi=200)
        plt.close(fig)
except Exception:
    pass

with open(os.path.join(OUTPUT_DIR, "00_run_log.txt"), "w", encoding="utf-8") as f:
    f.write("Run completed.\n")
